# Library

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-huggingface langchain-google-genai langsmith langchain-teddynote

In [35]:
from src.utils import show_stream

from dotenv import load_dotenv

import os
from datetime import datetime
from pydantic import BaseModel, Field
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_core.load import dumpd, dumps, load, loads
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.callbacks.manager import get_openai_callback
from langchain_community.chat_models import ChatOllama
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
    MaxMarginalRelevanceExampleSelector,
)
from langchain_core.output_parsers import (
    StrOutputParser,
    PydanticOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser,
)
from langchain.output_parsers.structured import (
    ResponseSchema,
    StructuredOutputParser,
)
from langchain.output_parsers.pandas_dataframe import PandasDataFrameOutputParser
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from langchain.output_parsers.retry import RetryOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

In [3]:
load_dotenv()

True

# LangChain

자연어 처리와 대화형 AI 애플리케이션을 구축하는 데 도움을 주는 프레임워크. <br>
여러 가지 NLP 모델과 데이터 소스를 쉽게 통합할 수 있도록 설계. <br>
복잡한 대화형 시스템을 개발하는 데 필요한 다양한 도구와 컴포넌트를 제공. <br>
대화형 애플리케이션, 챗봇, 자동화된 고객 서비스 솔루션 등을 보다 효율적으로 개발 가능. <br>

<br>

<font style="font-size:20px"> 특징 </font>

- 모델 통합: 다양한 언어 모델(예: GPT, Claude 등)을 쉽게 사용할 수 있도록 지원
- 데이터 소스 연결: 데이터베이스, API, 파일 시스템 등 여러 소스와 연결할 수 있는 기능 제공
- 체인 구성: 여러 작업을 순차적으로 연결하여 복잡한 프로세스를 만들 수 있는 체인 개념 도입
- 사용자 정의: 사용자 요구에 맞게 쉽게 확장하고 커스터마이즈 가능

## LangSmith

LangSmith: https://smith.langchain.com/ <br>

LLM 기반 애플리케이션의 모든 단계에 대한 올인원 개발 플랫폼. <br>
LLM 애플리케이션을 디버깅하고, 협업하며, 테스트하고, 모니터링 가능. <br>

<br>

<font style="font-size:20px"> 설정 </font>

.env파일에 아래 입력 <br>

```cmd
OPENAI_API_KEY=<api_key>
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_TRACING_V2=true
LANGCHAIN_API_KEY=<api-key>
LANGCHAIN_PROJECT=<project_name>
```

<br>

```python
from langchain_teddynote import logging

logging.langsmith("<project name>") # tracking on
logging.langsmith("<project name>", set_enable=False)   # tracking off
```

## OpenAI API

```python
llm = ChatOpenAI(
    temperature=0.1,            # 0.0 ~ 2.0
    model_name='gpt-4o-mini',   # model name
    # max_tokens=2048,      # 최대 토큰 수
)#.bind(logprobs=True)      # logprob 출력


# question
question = ''
print(f'answer: {llm.invoke(question)}')

# stream
question = ''
answer = llm.stream(question)

for token in answer:
    print(token.content, end='', flush=True)

# 토큰, 요금 사용량 추적
with get_openai_callback() as callback:
    result = llm.invoke(question)
    print(callback)
```

In [152]:
# ChatOpenAI 클래스의 인스턴스를 생성하여 LLM(대형 언어 모델)을 설정합니다.
llm = ChatOpenAI(
    # 생성되는 응답의 다양성을 조절하는 온도 설정
    temperature=1,  # 온도가 높을수록 응답이 더 창의적이고 예측 불가능해짐
    # 사용할 모델의 이름 설정
    model_name='gpt-4o-mini',  # 'gpt-4o-mini' 모델을 사용
)

In [153]:
# 질문을 정의합니다. 제주도의 관광 명소에 대한 정보 요청.
question = '제주도의 관광 명소에 대해서 알려줘'

# 정의한 질문을 LLM에 전달하여 응답을 받아옵니다.
response = llm.invoke(question)

# 응답의 내용을 출력합니다.
print(response.content)

제주도는 한국의 대표적인 관광지로, 다양한 자연경관과 문화유산이 매력적인 곳입니다. 아래는 제주도의 주요 관광 명소 몇 곳입니다.

1. **한라산**: 제주도의 중심에 위치한 한라산은 대한민국에서 가장 높은 산으로, 다양한 등산로와 아름다운 경관을 제공합니다. 특히, 계절마다 변화하는 자연의 모습을 즐길 수 있습니다.

2. **성산 일출봉**: UNESCO 세계자연유산으로 지정된 이곳은 일출을 보기 위해 많은 관광객이 찾는 장소입니다. 정상에서 바라보는 바다와 오름의 풍경이 매력적입니다.

3. **제주도 해변**: 협재 해수욕장, 중문 해수욕장, 함덕 해수욕장 등 다양한 아름다운 해변이 있어 수영, 해양 스포츠, 일몰 감상을 즐길 수 있습니다.

4. **万두(만장굴)**: 세계적으로 유명한 용암 동굴로, 길이가 약 7.4km에 이릅니다. 독특한 지형과 생태계를 관찰할 수 있는 장소입니다.

5. **김녕 해수욕장**: 푸른 바다와 하얀 모래사장이 어우러져 아름다운 경관을 자랑합니다. 특히, 수심이 얕아 가족 단위 방문객에게 인기가 많습니다.

6. **제주민속촌**: 제주도의 전통 민속문화를 체험할 수 있는 장소로, 다양한 전통 가옥과 문화유산이 전시되어 있습니다.

7. **오름(작은 화산)**: 제주도 전역에 분포하는 작은 화산체로 하이킹과 자연 관찰에 좋은 코스가 많습니다. 가장 유명한 오름으로는 성산 일출봉, 분화구가 있는 수월봉 등이 있습니다.

8. **제주 국제 공항**: 제주도에 들어오는 관문으로, 공항에서 가까운 큰 관광지들이 많아 편리합니다.

9. **용두암**: 바닷가에 위치한 자연 경관으로, 용의 머리를 형상화한 암석이 특징입니다. 특히 일몰 시간에 가면 아름다운 경관을 감상할 수 있습니다.

10. **우도**: 제주도의 한 작은 섬으로, 맑은 바다와 아름다운 경관, 그리고 다양한 해양 스포츠 프로그램이 인기를 끌고 있습니다.

이 외에도 제주도에는 수많은 관광 명소가 있으니, 방문 시 다양한 활동과 경험을 해보시길 권장합니다!

In [185]:
# LLM에 질문을 전달하여 스트리밍 방식으로 응답을 받아옵니다.
# 스트리밍 방식은 응답을 실시간으로 받으며, 점진적으로 결과를 출력할 수 있습니다.
response = llm.stream(question)

In [187]:
# 응답에서 각 토큰을 순회하며 처리합니다.
for token in response:
    # 각 토큰의 내용을 출력합니다. 
    # end=''는 출력 후 줄 바꿈을 방지하고, flush=True는 버퍼를 즉시 비워 실시간으로 출력하게 합니다.
    print(token.content, end='', flush=True)

제주도는 아름다운 자연 경관과 독특한 문화로 유명한 관광지입니다. 다음은 제주도의 주요 관광 명소입니다:

1. **한라산**: 제주도의 중앙에 위치한 한라산은 한국에서 가장 높은 산으로, 등산과 자연 경관을 즐길 수 있는 인기 있는 장소입니다.

2. **제주 해변**: 제주도에는 여러 아름다운 해변이 있습니다. 특히, 함덕 해변, 중문 해변, 협재 해변 등이 유명합니다. 이곳에서는 수영, 서핑, 일광욕 등을 즐길 수 있습니다.

3. **성산 일출봉**: UNESCO 세계자연유산으로 지정된 이곳은 일출 명소로 유명합니다. 봉우리에 올라가면 멋진 경치를 감상할 수 있습니다.

4. **만장굴**: 세계에서 가장 긴 용암동굴 중 하나로, 다양한 형태의 종유석과 석순을 볼 수 있습니다.

5. **우도**: 제주 동쪽에 위치한 작고 아름다운 섬으로, 자전거를 타고 여행하기에 좋은 곳입니다. 해변과 경치가 아름다워 많은 관광객들이 방문합니다.

6. **제주 민속촌**: 제주 전통 가옥과 문화를 체험할 수 있는 공간으로, 제주도의 역사와 문화를 이해하는 데 도움이 됩니다.

7. **천지연 폭포**: 아름다운 폭포로, 주위의 자연경관과 조화를 이루며 힐링을 원하는 관광객들에게 인기가 많습니다.

8. **섭지코지**: 아름다운 해안 경치와 함께 다양한 산책로가 있는 곳으로, 사진 촬영하기에 좋은 명소로 알려져 있습니다.

9. **상효원**: 다양한 식물과 꽃들로 가득한 정원으로, 편안한 산책을 즐기기에 좋은 장소입니다.

10. **제주 아쿠아플라넷**: 다양한 해양 생물을 관람할 수 있는 대형 아쿠아리움으로, 가족 단위 관광객에게 인기가 많습니다.

이외에도 제주도에는 다양한 식당, 카페, 기념품 가게 등이 있어 여행을 더욱 즐겁게 만들어 줍니다.

In [189]:
# 스트리밍 응답을 출력하는 함수 정의
def show_stream(response):
    # 응답에서 각 토큰을 순회하며 처리합니다.
    for token in response:
        # 각 토큰의 내용을 출력합니다. 
        # end=''는 출력 후 줄 바꿈을 방지하고, flush=True는 버퍼를 즉시 비워 실시간으로 출력하게 합니다.
        print(token.content, end='', flush=True)

In [190]:
# LLM에 질문을 전달하여 스트리밍 방식으로 응답을 받아옵니다.
response = llm.stream(question)

# 받아온 응답을 show_stream 함수를 사용하여 출력합니다.
show_stream(response)  # 스트리밍 응답을 실시간으로 출력

제주도는 아름다운 자연경관과 다양한 관광 명소로 유명한 한국의 대표적인 관광지입니다. 다음은 제주도의 주요 관광 명소입니다.

1. **한라산 국립공원**: 제주도의 중심에 위치한 한라산은 한국에서 가장 높은 산으로, 다양한 등산 코스와 아름다운 자연을 제공합니다. 사계절 내내 다른 매력을 지니고 있습니다.

2. **제주 올레길**: 제주도를 둘러싼 걷기 좋은 트레일 코스로, 다양한 경치를 즐기며 천천히 걸을 수 있습니다. 총 26개 코스가 있으며, 각기 다른 매력을 지니고 있습니다.

3. **성산 일출봉**: UNESCO 세계자연유산으로 등록된 성산 일출봉은 일출을 감상하기에 최적의 장소입니다. 독특한 형태의 화산체와 주변 바다 경관이 아름답습니다.

4. **우도**: 제주도에서 배를 타고 갈 수 있는 작은 섬으로, 아름다운 해변과 맑은 바다, 다양한 해양 스포츠를 즐길 수 있습니다.

5. **천지연폭포**: 제주시 근처에 위치한 폭포로, 아름다운 경치와 함께 시원한 물소리를 즐길 수 있습니다. 주변에는 산책로도 잘 조성되어 있습니다.

6. **제주 민속촌**: 제주도의 전통적인 문화와 생활상을 체험할 수 있는 장소로, 다양한 민속 자료와 건물들이 전시되어 있습니다.

7. **산방산과 송악산**: 두 개의 산 모두 독특한 형태로 유명하며, 각각의 정상에서 내려다보는 경치는 일품입니다. 특히 산방산에서는 근처의 바다와 해변을 함께 감상할 수 있습니다.

8. **계란밭**: 유명한 소금밭에서의 소금작업 체험과 함께 제주도의 전통적인 농업을 배울 수 있는 체험 공간입니다.

9. **용머리해안**: 용의 머리 모양을 한 해안 절벽으로, 자연의 아름다움을 감상할 수 있는 좋은 장소입니다.

10. **제주 테지움**: 다양한 테마의 테마파크로, 가족 단위 관광객에게 인기 있는 곳입니다.

제주도는 이 외에도 많은 관광 명소와 액티비티를 제공하니, 여행 계획 시 다양한 경험을 고려해보시기 바랍니다!

In [194]:
# 제주도의 관광 명소에 대한 질문을 정의합니다.
# 요청 형식은 JSON 형식의 리스트로, 각 요소는 관광명소와 그에 대한 설명을 포함합니다.
question = '''
제주도의 관광 명소에 대해서 알려줄래?

관광 명소는 다음과 같은 형식으로 주면 좋겠어.
{관광명소: 내용}의 json 형식으로 쌓인 list로 돌려줘
key가 관광명소이고 value가 관련 내용이야
'''

# LLM에 질문을 전달하여 스트리밍 방식으로 응답을 받아옵니다.
response = llm.stream(question)

# 받아온 응답을 show_stream 함수를 사용하여 출력합니다.
show_stream(response)  # 스트리밍 응답을 실시간으로 출력

물론입니다! 제주도의 관광 명소에 대한 정보를 JSON 형식으로 제공해드리겠습니다.

```json
[
    {"관광명소": "한라산", "내용": "한라산은 제주도의 중앙에 위치한 한국에서 가장 높은 산으로, 아름다운 자연 경관과 다양한 생태계를 자랑합니다."},
    {"관광명소": "제주 올레길", "내용": "제주 올레길은 제주 해안을 따라 이어지는 도보 여행길로, 아름다운 바다 절경과 풍경을 즐길 수 있는 코스입니다."},
    {"관광명소": "성산 일출봉", "내용": "성산 일출봉은 UNESCO 세계자연유산에 등재된 화산구로, 일출을 보기 위한 명소로 유명하며, 정상에서 바라보는 경관이 장관입니다."},
    {"관광명소": "천지연 폭포", "내용": "천지연 폭포는 제주에서 가장 유명한 폭포 중 하나로, 맑은 물줄기와 아름다운 주변 자연경관이 어우러져 있습니다."},
    {"관광명소": "만장굴", "내용": "만장굴은 세계에서 가장 긴 용암동굴 중 하나로, 독특한 지질 구조와 다양한 형태의 용암 암석을 관찰할 수 있습니다."},
    {"관광명소": "제주 민속촌", "내용": "제주 민속촌은 제주도의 전통적인 농촌 생활과 문화를 체험할 수 있는 야외 박람회로, 조선시대의 제주 마을을 재현하고 있습니다."},
    {"관광명소": "협재해수욕장", "내용": "협재해수욕장은 백사장과 맑은 바다로 유명한 해수욕장으로, 해변에서 여러 수상 스포츠와 레저 활동을 즐길 수 있습니다."},
    {"관광명소": "섬속의 섬 우도", "내용": "우도는 제주도에서 배로 15분 거리에 위치한 작은 섬으로, 아름다운 해변과 독특한 풍경으로 유명합니다."},
    {"관광명소": "곽지해수욕장", "내용": "곽지해수욕장은 잔잔한 바다와 고운 모래사장이 특징으로, 가족 단위 여행객들에게 인기가 많습니다."},
    {"관광명소": "용두암", "내용": "용두암은 독특한 용의 머리 형상을 가진 바위로, 제주도의 랜드마크 중 하나이며, 아

## LangChain Expression Language (LECL)

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FrmjEl%2FbtsH2HENPoU%2Fot1Uqpc1z0FxlLjduntoNK%2Fimg.png" width="800" height="300">

체인을 쉽게 구성할 수 있는 방법. <br>
프로토타입을 코드 변경 없이 생산에 배포할 수 있도록 설계. <br>
간단한 프롬프트 + LLM 체인부터 복잡한 체인까지 지원. <br>
LCEL의 구성요소는 파이프 기호(|)로 구분된 순서로 배치. <br>
체인은 왼쪽에서 오른쪽으로 실행. <br>

<br>

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbDiBzx%2FbtsH2D3u5lv%2FRCoYXHK9fa2adzxl9xH5L0%2Fimg.png" width="800" height="400">

<br>
<br>

<font style="font-size:20px"> 특징 <font> <br>

- 1급 스트리밍 지원: 빠른 응답 시간을 제공하며, LLM에서 출력 파서로 직접 스트리밍 <br>
    1급 스트리밍: 정보가 생성되는 직시 사용자에게 실시간으로 전달되는 방식
- 비동기 지원: 동기 및 비동기 API 모두에서 호출 가능하여, 프로토타입과 생산 환경에서 동일한 코드 사용 가능
- 최적화된 병렬 실행: LCEL 체인의 병렬 실행이 가능한 단계는 자동으로 병렬로 처리되어 지연 시간을 최소화
- 재시도 및 대체: LCEL 체인의 어떤 부분에 대해서도 재시도와 대체를 구성할 수 있어 신뢰성 향상
- 중간 결과 접근: 복잡한 체인에서는 최종 출력 전에 중간 결과에 접근할 수 있어 디버깅 및 사용자에게 진행 상황을 알리는 데 유용
- 입력 및 출력 스키마: Pydantic 및 JSONSchema를 기반으로 입력 및 출력 스키마 제공
- LangSmith 추적: 체인이 복잡해짐에 따라 모든 단계가 자동으로 LangSmith에 기록되어 관찰과 디버깅 용이
- LangServe 배포: LangServe를 통해 쉽게 배포 가능

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> template = '{country}의 역사에 대해서 쉽게 알려줘.'
> model = ChatOpenAI()
> prompt = PromptTemplate.from_template(template)
> output_parser = StrOutputParser()
> 
> # basic chain
> chain1 = prompt | model | output_parser
> chain2 = (
>   PromptTemplate.from_template('30년 전 {country}의 경제는 어땠는지 알려줄래?')
>   | model
>   | StrOutputParser()   
> )
> 
> # 실행예
> subject1 = {'country': '고조선 시대'}
> subject2 = {'country': '전국시대'}
> chain1.invoke(subject1) # invoke: 입력 값을 전달, 입력은 dictionary 권장
> chain1.stream(subject2) # stream: 스트리밍 출력
> 
> chain1.batch([subject1, subject2], config={'max_concurrency': 5}) # batch 단위 처리, max_concurrency: 최대 동시 처리 수
> parallel = RunnableParallel(history=chain1, economy=chain2)
> parallel.batch([{'country': '한국'}, {'country': '일본'}])
> 
> abatch_result = chain1.abatch([subject1, subject2], config={'max_concurrency': 5})
> result = await abatch_result  # 비동기 작업이 완료될 때까지 대기
>
> async for token in chain1.stream({'country': '미국'}):
>   print(token, end='', flush=True)
>
>
> # Runnable
> RunnablePassthrough().invoke(1)
> RunnablePassthrough().invoke({'country': '한국'})
>
> prompt = PromptTemplate.from_template("{country}의 사회 문화에 대해서 설명해줄래?") 
> runnable_chain = {'country': RunnablePassthrough()} | prompt | ChatOpenAI()
> runnable_chain.invoke('한국')
>
> RunnablePassthrough.assign(dynasty=lambda x: '조선' if x['country']=='한국' else x['country']).invoke({'country': '한국'})
>
> prompt = PromptTemplate.from_template(
>     "100년 전 {today}에 발생한 유명한 사건 {n}개를 나열. 발생일시를 생년월일로 표기."
> )
> def today(_):
>     return datetime.today().strftime('%Y-%m-%d')
> chain = (
>     {"today": RunnableLambda(today), "n": RunnablePassthrough()}
>     | prompt
>     | llm
>     | StrOutputParser()
> )
> ```

<br>

<font style="font-size:18px"> 표준 </font> <br>
- stream: 응답의 chunk 스트리밍
- invoke: 입력에 대해 체인 호출
- batch: 입력 목록에 대해 체인 호출

<br>

<font style="font-size:18px"> 비동기 </font>
- astream: 비동기적 응답의 chunk 스트리밍
- ainvoke: 비동기적 입력에 대한 체인 호출
- abatch: 비동기적 입력 목록에 대한 체인 호출
- astream_log: 중간 단계 스트리밍

In [197]:
# 질문 템플릿을 정의합니다. {country}는 변수가 들어갈 자리입니다.
template = "{country}의 현재 GDP는 어떻게 되니?"

# 템플릿을 사용하여 PromptTemplate 객체를 생성합니다.
# 이 객체는 나중에 변수를 넣어 실제 질문을 만들 때 사용됩니다.
prompt = PromptTemplate.from_template(template)

# 생성된 prompt 객체를 출력합니다.
prompt


PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 현재 GDP는 어떻게 되니?')

In [198]:
# ChatOpenAI 클래스의 인스턴스를 생성하여 LLM(대형 언어 모델)을 설정합니다.
llm = ChatOpenAI(
    # 생성되는 응답의 다양성을 조절하는 온도 설정
    temperature=1,  # 온도가 높을수록 응답이 더 창의적이고 예측 불가능해짐
    # 사용할 모델의 이름 설정
    model_name='gpt-4o-mini'  # 'gpt-4o-mini' 모델을 사용
)


In [213]:
# 정의된 prompt와 LLM을 결합하여 체인을 생성합니다.
# 이 체인은 템플릿에 변수를 넣고, 그 결과를 LLM에 전달하는 역할을 합니다.
chain = prompt | llm

# 체인을 사용하여 '한국'이라는 변수를 포함한 질문을 실행합니다.
response = chain.invoke({'country': '한국'})

# 생성된 응답을 출력합니다.
response

AIMessage(content='2023년 한국의 GDP에 대한 정확한 데이터를 제공할 수는 없습니다. 그러나 2021년과 2022년의 GDP 통계를 바탕으로 추세를 말씀드릴 수 있습니다. 한국의 GDP는 지속적으로 성장하고 있으며, 2022년의 명목 GDP는 약 1.8조 달러를 넘었습니다. 2023년의 예상 GDP에 대한 정보는 한국은행이나 국제통화기금(IMF) 등의 공식 기관에서 확인할 수 있습니다. 최신 정보를 원하신다면, 해당 기관의 웹사이트나 최근 경제 보고서를 참고하시기 바랍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 16, 'total_tokens': 143, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7693ae462b', 'finish_reason': 'stop', 'logprobs': None}, id='run-63722294-0d13-4abd-879e-7a9a47b83220-0', usage_metadata={'input_tokens': 16, 'output_tokens': 127, 'total_tokens': 143, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [204]:
# 문자열 출력 형식을 처리하기 위한 파서를 생성합니다.
parser = StrOutputParser()

# 정의된 prompt와 LLM, 그리고 파서를 결합하여 체인을 생성합니다.
# 이 체인은 템플릿에 변수를 넣고, LLM에서 응답을 받아 파서로 처리하는 역할을 합니다.
chain = prompt | llm | parser

In [206]:
# 체인을 사용하여 '한국'이라는 변수를 포함한 질문을 실행합니다.
# LLM이 응답한 결과는 StrOutputParser를 통해 처리됩니다.
response = chain.invoke({'country': '한국'})

# 생성된 응답을 출력합니다.
response

'2023년 기준으로 한국의 GDP(국내총생산)는 약 1.8조 달러(USD) 정도로 추정됩니다. 하지만 정확한 수치는 변동성이 클 수 있으므로, 최신 자료를 참고하는 것이 중요합니다. 더 구체적인 정보나 최신 통계를 원하신다면, 한국은행이나 경제 관련 공식 기관의 발표를 확인해 보시는 것을 추천드립니다.'

In [215]:
# example
## 날짜(yyyy-mm-dd)를 입력받아 해당 날짜에 일어난 역사적 사건을 return받는 chain 구성
## prompt 정의
template = '{date}에 일어난 역사적 사건에 대해 알려줘'
prompt = PromptTemplate.from_template(template)

## model 정의
model = ChatOpenAI(
    temperature=1,
    model='gpt-4o-mini'
)

## parser 정의
parser = StrOutputParser()

## chain 정의
chain = prompt | model | parser

## 입력
print(chain.invoke({'date': "1924-10-23"}))

1924년 10월 23일에 발생한 특정한 역사적 사건에 대한 정보는 명확히 기록되어 있지 않습니다. 그러나 1924년은 여러 중요한 역사적 사건들이 있었던 시기였습니다. 예를 들어, 이 해에는 제1차 세계대전 이후의 세계 정치 및 경제적 변화가 두드러졌으며, 다양한 사회적 움직임과 문화적 변혁이 있었습니다.

만약 더 구체적인 사건이나 주제가 필요하다면, 해당 날짜와 관련된 다른 사건이나 더 넓은 맥락에 대해 알려드릴 수 있습니다.


In [219]:
# example
## 국가와 날짜와 요일을 입력받아 해당 국가의 해당 날짜의 해당 요일에 일어났던 사건에 대해 return
template = '{country}에서 {date}의 {day}에 일어난 역사적인 사건에 대해서 알려줄래?'

## prompt 정의
prompt = PromptTemplate.from_template(template)

## model 정의
model = ChatOpenAI(
    temperature=1,
    model_name='gpt-4o-mini'
)

## parser 정의
parser = StrOutputParser()

## chain 정의
chain = prompt | model | parser

## 입력
with get_openai_callback() as callback:
    chain.invoke({'country': '한국', 'date': '1950년', 'day': '일'})
    print(callback)

Tokens Used: 238
	Prompt Tokens: 27
	Completion Tokens: 211
Successful Requests: 1
Total Cost (USD): $0.00013064999999999998


In [224]:
# 배치 처리를 위한 질문 템플릿을 정의합니다.
template = '{country}의 역사에 대해서 알려줄래?'
prompt = PromptTemplate.from_template(template)

# LLM 설정: 생성되는 응답의 다양성을 조절하는 온도와 모델 이름을 설정합니다.
model = ChatOpenAI(
    temperature=1,  # 온도가 높을수록 응답이 더 창의적이고 예측 불가능해짐
    model='gpt-4o-mini',  # 사용할 모델 설정
)

# 문자열 출력 형식을 처리하기 위한 파서를 생성합니다.
parser = StrOutputParser()

# prompt, LLM, 파서를 결합하여 체인을 생성합니다.
chain = prompt | model | parser

# for country in ('한국', '일본', '중국', '러시아', '인도'):
#     chain.invoke({'country': country})

# 여러 나라에 대한 역사 정보를 배치로 요청합니다.
# 최대 동시 실행 수를 3으로 설정하여 성능을 최적화합니다.
responses = chain.batch([
    {'country': '한국'},
    {'country': '인도'},
    {'country': '중국'},
    {'country': '일본'},
    {'country': '러시아'},
],
config={'max_concurrency': 3})  # 동시에 실행할 요청 수 설정

In [223]:
# 여러 나라에 대한 역사 정보를 개별적으로 요청하기 위해 반복문을 사용합니다.
for country in ('한국', '일본', '중국', '러시아', '인도'):
    # 각 나라에 대해 체인을 사용하여 질문을 실행합니다.
    chain.invoke({'country': country})  # 각 나라의 역사에 대한 응답을 요청

In [222]:
responses

['한국의 역사는 매우 오래되고 복잡하며, 다음과 같은 주요 시기로 나눌 수 있습니다.\n\n### 고대\n- **선사 시대**: 한국의 선사 시대는 기원전 약 7000년 전부터 시작됩니다. 이 시기에는 주로 수렵과 채집을 통해 생계를 이어갔습니다.\n- **삼국 시대 (기원전 1세기 - 기원후 7세기)**: 고구려, 백제, 신라 세 나라가 각축을 벌였습니다. 각 나라는 독립적인 문화와 정치체계를 갖추었으며, 이 시기의 중요한 사건 중 하나로는 신라의 통일이 있습니다.\n\n### 통일 신라 및 발해\n- **통일 신라 (668-935)**: 신라가 고구려와 백제를 정복하고 한반도를 통일합니다. 이 시기에 불교가 융성하고 문화, 예술이 발전했습니다.\n- **발해 (698-926)**: 고구려의 유민들이 세운 왕국으로, 북부 지역에서 강력한 세력을 구축했습니다. 발해는 무역과 문화교류가 활발했습니다.\n\n### 고려 시대 (918-1392)\n고려는 왕건이 세운 국가로, 중앙집권적 체제를 확립하고 불교가 국교로 자리 잡습니다. 고려의 중요한 업적 중 하나는 두 차례의 외침(몽골 invasions)에도 불구하고 국가를 유지하고 발전시킨 것입니다.\n\n### 조선 시대 (1392-1910)\n- **조선은 이성계가 세운 왕조로, 유교를 국교로 삼아 중앙 집권 체제를 강화했습니다.**\n- **16세기**에는 임진왜란(일본의 침략)이 발생했으며, 이로 인해 대규모 전투와 피해가 있었습니다.\n- 조선 후기에는 서양과의 교류가 점차 증가하며, 일본의 침략과 내부의 혼란이 이어졌습니다.\n\n### 일제 강점기 (1910-1945)\n- 일본 제국에 의해 한반도가 강제로 식민지화 되면서, 한국인들은 문화적, 정치적 억압을 받았습니다. 이 시기에 많은 독립운동이 일어났습니다.\n\n### 현대 (1945-현재)\n- **해방 (1945)**: 제2차 세계 대전의 종전과 함께 한국은 해방되지만, 남북으로 분단됩니다.\n- **한국 전쟁 (1950-1953)**: 전쟁으

In [240]:
# 템플릿 정의: 특정 국가와 날짜에 발생한 역사적 사건을 요청하는 형식
template = '{country}에서 {date} {day}에 발생한 역사적 사건에 대해서 알려줄래?'

# PromptTemplate 객체 생성: 입력 변수로 'country'를 사용하고, 'day'와 'date'는 고정값으로 설정
prompt = PromptTemplate(
    template=template,
    input_variables=['country'],  # 사용자 입력 변수
    partial_variables={            # 고정 변수
        'day': '월요일',           # 고정된 요일
        'date': '2008년',          # 고정된 연도
    }
)

# ChatOpenAI 모델 설정: 온도와 모델 종류 지정
model = ChatOpenAI(
    temperature=1,                # 창의적인 응답을 위해 높은 온도 설정
    model='gpt-4o-mini',         # 사용할 모델 이름
)

# 출력 파서를 설정: 문자열 출력을 처리하는 파서
parser = StrOutputParser()

# 전체 체인 구성: 템플릿, 모델, 파서를 연결하여 입력에 따라 결과 생성
chain = prompt | model | parser

# 한국에서 2015년에 발생한 사건에 대한 정보를 요청
print(chain.invoke({'country': '한국', 'date': '2015년'}))

2015년 1월 5일 월요일, 한국에서는 '삼일절 기념식'을 포함한 여러 역사적 사건과 관련된 다양한 프로그램이 진행되었습니다. 그러나 특정 날짜에 대한 역사적 사건은 상대적으로 제한적이므로, 2015년에 발생한 주요 사건으로는 2015년 메르스(중동호흡기증후군) 사태가 있습니다. 이 사태는 한국에서 발생하면서 엄청난 사회적 파장을 일으켰고, 보건 당국의 대응과 국민의 경각심을 촉발하는 계기가 되었습니다.

이외에도 2015년에는 박근혜 정부 시절의 여러 사회·정치적 사건들이 있었지만, 특정한 역사적 사건은 개인의 관점에 따라 다를 수 있습니다. 특정 날짜의 역사적 사건에 대한 더 구체적인 정보가 필요하시다면 요청하시면 도와드리겠습니다!


In [242]:
# 여러 요청을 동시에 처리하기 위해 batch 메서드를 사용
# 각각의 요청에 대해 국가와 날짜 정보를 포함한 리스트를 전달
results = chain.batch([
    {'country': '한국', 'date': '2015년'},  # 첫 번째 요청: 한국, 2015년
    {'country': '한국', 'date': '2015년'},  # 두 번째 요청: 한국, 2015년
])

# 결과를 출력
print(results)  # 배치 요청에 대한 결과를 출력

["2015년 1월 19일 월요일, 한국에서 발생한 notable한 사건 중 하나는 '사드' 배치와 관련된 논란이 본격화된 시기였습니다. 하지만 이 사건은 주로 2016년 이후로 더욱 주목받게 되었고, 2015년 1월 자체는 특별히 큰 역사적 사건으로 기록되지는 않았습니다.\n\n2015년 자체는 한국 사회에서 여러 가지 사건과 이슈가 있었는데, 예를 들어 세월호 사고와 관련된 논의가 계속 이어졌고, 정치적 이슈와 북한 관련 사안 등도 계속해서 뉴스에 오르내렸습니다. 추가적으로, 2015년에는 '하반기 아시아 경기 대회'와 같은 큰 행사가 있었지만 이는 주로 2014년에 대한 준비 작업의 연장선에 있었던 이벤트들입니다.\n\n정확한 사건을 원하는 경우에 특정한 날짜나 주제를 제공해주시면 더 자세히 조사해 드릴 수 있습니다.",
 '2015년 4월 27일 월요일은 한국에서 여러 가지 사건이 있었던 날이지만, 특히 주목할 만한 사건 중 하나는 세월호 참사와 관련된 이슈들입니다. 2014년 4월 16일에 발생한 세월호 침몰 사건은 한국 사회에 큰 충격을 주었고, 그 후속 처리가 2015년에도 여러 가지 논란과 사회적 논의를 불러일으켰습니다. \n\n2015년에는 세월호 유가족들이 진상규명 및 책임자 처벌을 요구하는 집회와 행사를 지속적으로 열었고, 당시 여론과 정치적 갈등이 심화되었습니다. 이와 같은 지속적인 움직임은 한국 사회의 안전 문제와 정부의 책임 소재에 대한 논의를 촉발했습니다.\n\n따라서, 2015년 봄은 한국 사회에 있어서 세월호의 기억과 그로 인한 사회적 변화가 중요한 시기였다고 할 수 있습니다. 세월호 참사는 한국 사회 전반에 걸쳐 많은 영향을 미친 사건으로 여전히 기억되고 있습니다.']

In [249]:
# example
# 국가 정보와 수도를 입력받아 해당 수도에 몇 명의 인구가 있는지 출력
# 기본 연도는 2010년, 그러나 유저의 요청으로 연도 변경이 가능해야 함
# 최소 10개 이상의 값을 받아야 하며, 시간을 최대한 적게 써야 함
# 한 번에 최대 요청할 수 있는 횟수는 5회

# 템플릿 정의: 특정 연도, 국가, 수도에 대한 인구를 요청하는 형식
template = '{year}년 {country} 국가의 {capital}에 몇 명의 인구가 있는지 알려줄래? 모르면 솔직하게 모른다고 얘기해라.'

# PromptTemplate 객체 생성: 'country'와 'capital'을 입력 변수로 사용하고 'year'는 고정값으로 설정
prompt = PromptTemplate(
    template=template,
    input_variables=['country', 'capital'],  # 사용자 입력 변수
    partial_variables={                        # 고정 변수
        'year': '2010'                        # 고정된 연도
    }
)

# ChatOpenAI 모델 설정: 사용할 모델과 온도 설정
model = ChatOpenAI(
    model='gpt-4o-mini',                      # 사용할 모델 이름
    temperature=1,                            # 창의적인 응답을 위해 높은 온도 설정
)

# 출력 파서를 설정: 문자열 출력을 처리하는 파서
parser = StrOutputParser()

# 전체 체인 구성: 템플릿, 모델, 파서를 연결
chain = prompt | model | parser

# 여러 요청을 동시에 처리하기 위해 batch 메서드를 사용
response = chain.batch([
    {'country': '한국', 'capital': '서울'},       # 첫 번째 요청: 한국, 서울
    {'country': '일본', 'capital': '도쿄'},       # 두 번째 요청: 일본, 도쿄
    {'country': '중국', 'capital': '베이징'},      # 세 번째 요청: 중국, 베이징
    {'country': '방글라데시', 'capital': '다카'},  # 네 번째 요청: 방글라데시, 다카
    {'country': '베트남', 'capital': '하노이'},    # 다섯 번째 요청: 베트남, 하노이
    {'country': '이란', 'capital': '테헤란'},      # 여섯 번째 요청: 이란, 테헤란
    {'country': '인도', 'capital': '뉴델리'},      # 일곱 번째 요청: 인도, 뉴델리
    {'country': '미국', 'capital': '워싱턴DC'},    # 여덟 번째 요청: 미국, 워싱턴DC
    {'country': '싱가포르', 'capital': '싱가포르'},  # 아홉 번째 요청: 싱가포르, 싱가포르
    {'country': '요르단', 'capital': '암만'},      # 열 번째 요청: 요르단, 암만
    ],
    config={                                   # 배치 요청 구성
        'max_concurrency': 5,                   # 최대 동시 요청 수를 5로 설정
    }
)

# 결과를 출력
print(response)  # 배치 요청에 대한 결과를 출력

['2010년 한국의 서울 인구는 약 10,440,000명(1천만 명) 정도였습니다. 이 수치는 2010년 인구 센서스를 기준으로 한 데이터입니다. 더 구체적인 정보가 필요하면 말씀해 주세요!',
 '2010년 일본 도쿄의 인구는 약 1,300만 명 정도였습니다. 그러나 정확한 수치는 연례 인구 조사나 통계에 따라 다를 수 있습니다. 일본의 도쿄도는 일본의 수도로, 인구 밀도가 매우 높은 지역 중 하나입니다. 더 구체적인 정보가 필요하면 해당 연도의 공식 통계 자료를 확인하는 것이 좋습니다.',
 '2010년 중국 베이징의 인구는 약 1,600만 명 정도였습니다. 이는 2010년 인구 센서스에 기반한 수치입니다. 정확한 인구 수는 센서스 결과에 따라 다소 차이가 있을 수 있습니다.',
 '2010년 방글라데시 다카의 인구는 대략 1,000만 명 정도로 추정됩니다. 하지만 정확한 수치는 인구 조사와 다양한 요인에 따라 달라질 수 있습니다. 보다 구체적인 수치는 정부의 통계 자료나 공식 발표를 참고하는 것이 좋습니다.',
 '2010년 하노이의 인구는 약 7백만 명 정도였습니다. 하지만 정확한 숫자는 여러 출처에 따라 다를 수 있으므로 참고용으로만 사용하시기 바랍니다.',
 '2010년 이란의 테헤란 인구는 약 8백만 명 정도로 추정됩니다. 하지만 정확한 숫자는 시기나 출처에 따라 다를 수 있으니 참고하시기 바랍니다.',
 '2010년 뉴델리의 인구는 약 1,800만 명으로 추정됩니다. 그러나 정확한 수치는 인구 조사 방법이나 시점에 따라 다를 수 있습니다. 추가적으로 구체적인 수치를 원하시면 인도 정부의 공식 인구 조사 자료를 참고하는 것이 좋습니다.',
 '2010년 기준으로 워싱턴 D.C.의 인구는 약 601,723명이었습니다. 더 궁금한 점이 있으면 말씀해 주세요!',
 '2010년 싱가포르의 인구는 약 5백만 명 정도였습니다. 정확한 수치는 통계청의 자료에 따라 조금 다를 수 있지만, 대체로 이 수치가 일반적으로 알려진 인구 수입니다.',
 '2010년 

In [255]:
## 비동기
async for token in chain.astream({'capital': '서울', 'country': '한국'}):
    print(token, end='', flush=True)

2010년 기준으로 서울의 인구는 약 10,440,000명이었습니다. 이는 2010년 인구주택총조사 결과를 바탕으로 한 수치입니다. 더 궁금한 점이 있으면 말씀해 주세요!

In [257]:
# 비동기 배치 요청: 여러 국가와 수도에 대한 인구 정보를 요청
response = await chain.abatch([
    {'country': '한국', 'capital': '서울'},       # 첫 번째 요청: 한국, 서울
    {'country': '일본', 'capital': '도쿄'},       # 두 번째 요청: 일본, 도쿄
    {'country': '중국', 'capital': '베이징'},      # 세 번째 요청: 중국, 베이징
    {'country': '방글라데시', 'capital': '다카'},  # 네 번째 요청: 방글라데시, 다카
    {'country': '베트남', 'capital': '하노이'},    # 다섯 번째 요청: 베트남, 하노이
    {'country': '이란', 'capital': '테헤란'},      # 여섯 번째 요청: 이란, 테헤란
    {'country': '인도', 'capital': '뉴델리'},      # 일곱 번째 요청: 인도, 뉴델리
    {'country': '미국', 'capital': '워싱턴DC'},    # 여덟 번째 요청: 미국, 워싱턴DC
    {'country': '싱가포르', 'capital': '싱가포르'},  # 아홉 번째 요청: 싱가포르, 싱가포르
    {'country': '요르단', 'capital': '암만'},      # 열 번째 요청: 요르단, 암만
    ],
    config={                                   # 배치 요청 구성
        'max_concurrency': 5,                   # 최대 동시 요청 수를 5로 설정
    }
)

# 결과를 출력
print(response)  # 비동기 배치 요청에 대한 결과를 출력


In [274]:
# 템플릿 정의: 특정 국가의 현재 경제 상황에 대한 질문
template = '{country}의 현재 경제 상황에 대해서 알려줄래?'

# PromptTemplate 객체 생성: 주어진 템플릿을 기반으로 입력 변수를 설정
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 모델 설정: 사용할 모델 이름과 온도 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,              # 창의적인 응답을 위해 높은 온도 설정
)

# 출력 파서를 설정: 모델의 출력을 문자열 형태로 처리하는 파서
parser = StrOutputParser()


In [284]:
# 체인 구성: 입력 변수를 처리하기 위해 RunnablePassthrough를 사용
chain = (
    {'country': RunnablePassthrough()}  # 'country' 입력을 그대로 통과시키는 runnable
    | prompt                            # 템플릿을 사용하여 질문 형성
    | model                             # 모델을 호출하여 응답 생성
    | parser                            # 응답을 처리하는 파서
)

# 체인에 '한국'을 입력하여 호출
response = chain.invoke('한국')  # '한국'의 경제 상황 요청

# 결과를 출력
print(response)  # 모델의 응답 출력


'2023년 기준으로 한국의 경제 상황은 여러 가지 요인에 의해 영향을 받고 있습니다. 다음은 주요 요소들입니다:\n\n1. **경제 성장률**: 한국의 GDP 성장률은 글로벌 경제 상황과 무역 환경에 크게 영향을 받고 있습니다. 세계 경제 불확실성, 특히 미국과 중국의 경제 상황, 공급망 문제 등이 한국 경제에 미치는 영향을 고려해야 합니다.\n\n2. **수출과 무역**: 한국은 경제의 상당 부분이 수출에 의존하고 있어, 반도체, 자동차, 전자제품 등의 주요 수출 품목의 수요가 중요합니다. 2023년에는 글로벌 수요 회복으로 인해 수출이 증가할 것으로 예상되지만, 통상마찰이나 보호무역주의 등 외부 요인이 변수로 작용할 수 있습니다.\n\n3. **물가 상승**: 전 세계적으로 인플레이션이 문제가 되고 있는 가운데, 한국도 물가 상승 압력을 느끼고 있습니다. 에너지 및 식료품 가격 상승이 물가에 영향을 미치고 있으며, 이는 소비자 물가와 생활비에 부담을 주고 있습니다.\n\n4. **금리**: 한국은행의 금리 정책도 경제에 큰 영향을 미치고 있습니다. 금리 인상과 인하 결정은 소비와 투자 활동에 직접적인 영향을 주기 때문에, 유동성을 관리하는 것이 중요한 상황입니다.\n\n5. **고용 시장**: 고용 상황은 회복세를 보이고 있으나, 청년층 실업률과 장기 실업 문제는 여전히 해결해야 할 과제로 남아 있습니다. 정부의 일자리 정책과 기업의 채용 상황이 중요한 요소로 작용합니다.\n\n6. **정책 및 재정**: 정부의 경제 정책, 재정 지원, 규제 완화 등이 경제 상황에 큰 영향을 줄 수 있습니다. 특히, 혁신 산업과 디지털 전환을 위한 정책이 중요하게 다뤄지고 있습니다.\n\n이러한 요소들을 종합적으로 고려할 때, 한국 경제는 외부 환경에 대한 민감성을 보이며 회복과 성장을 모색하고 있지만, 지속적인 도전과제가 존재합니다.'

In [4]:
# RunnablePassthrough를 사용하여 입력을 처리하고 'capital' 값을 할당
response = (
    RunnablePassthrough()  # 입력을 그대로 통과시키는 runnable
    .assign(capital=lambda x: '서울' if x['country'] == '한국' else '모름')  # 조건에 따라 수도 할당
    .invoke({'country': '한국'})  # '한국'이라는 입력으로 invoke 호출
)

# 결과를 출력
print(response)  # 'capital' 값을 포함한 결과 출력


{'country': '한국', 'capital': '서울'}

In [ ]:
from datetime import datetime  # datetime 모듈 임포트

# 템플릿 정의: 50년 전의 날짜와 사건 수를 포함한 요청 템플릿
template = '50년 전 {today}에 발생한 역사적 사건 {n}개를 나열해봐. 발생일시는 yyyy-mm-dd 형식으로 출력'
prompt = PromptTemplate.from_template(template)  # 템플릿을 기반으로 PromptTemplate 객체 생성

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,              # 창의적인 응답을 위해 높은 온도 설정
)

# 출력 파서를 설정: 모델의 출력을 문자열 형태로 처리하는 파서
parser = StrOutputParser()

# 현재 날짜를 'yyyy-mm-dd' 형식으로 반환하는 함수
def get_today(_):
    return datetime.today().strftime('%Y-%m-%d')

# 전체 체인 구성: 입력 변수를 처리하는 구성 요소들을 연결
chain = (
    {
        'today': RunnableLambda(get_today),  # 현재 날짜를 얻는 runnable
        'n': RunnablePassthrough(),           # 사건 수를 그대로 통과시키는 runnable
    }
    | prompt                            # 템플릿을 사용하여 질문 형성
    | model                             # 모델을 호출하여 응답 생성
    | parser                            # 응답을 처리하는 파서
)

# 사건 수를 5로 설정하여 체인을 호출
print(chain.invoke(5))  # 50년 전 사건 5개 요청 결과 출력

In [24]:
# 현재 날짜를 'yyyy-mm-dd' 형식으로 반환하는 함수
def get_today(_):
    return datetime.today().strftime('%Y-%m-%d')

# 입력을 그대로 통과시키면서 'today' 값을 할당하는 Runnable 구성
response = (
    RunnablePassthrough()  # 입력을 변환 없이 그대로 통과시키는 runnable
    .assign(today=RunnableLambda(get_today))  # 'today' 필드를 현재 날짜로 할당
    .invoke({'n': 5})  # 사건 수를 5로 설정하여 invoke 호출
)

# 결과를 출력
print(response)  # 'n'과 'today'를 포함한 결과 출력

{'n': 5, 'today': '2024-10-23'}

In [27]:
## parallel
# 역사에 대한 요청 템플릿 정의
template1 = '{country}의 역사에 대해서 간략하게 알려줘'
# 경제 구조에 대한 요청 템플릿 정의
template2 = '{country}의 경제 구조에 대해서 간략하게 알려줘'

# 각 템플릿에 대해 PromptTemplate 객체 생성
prompt1 = PromptTemplate.from_template(template1)
prompt2 = PromptTemplate.from_template(template2)

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,              # 창의적인 응답을 위해 높은 온도 설정
)

# 출력 파서를 설정: 모델의 출력을 문자열 형태로 처리하는 파서
parser = StrOutputParser()

# 각 체인 구성: 역사와 경제에 대한 요청을 처리
chain1 = prompt1 | model | parser  # 역사 체인
chain2 = prompt2 | model | parser  # 경제 체인

# RunnableParallel을 사용하여 두 개의 체인을 병렬로 처리
parallel = RunnableParallel(history=chain1, economy=chain2)

# '한국'에 대한 정보를 병렬로 요청
response_kr = parallel.invoke({'country': '한국'})  # 한국의 역사와 경제 정보 요청
print(response_kr)  # 결과 출력

# '한국'과 '일본'에 대한 정보를 배치로 요청
response_batch = parallel.batch([{'country': '한국'}, {'country': '일본'}])  # 두 국가에 대한 요청
print(response_batch)  # 결과 출력

[{'history': "한국의 역사는 수천 년에 걸쳐 다양한 문화와 사건들이 얽혀 있습니다. 간략히 정리하면 다음과 같습니다.\n\n1. **고대사**: 한국의 역사는 기원전 2333년 고조선의 건국으로 시작되었다고 전해집니다. 이후 고조선은 한반도의 다른 고대 국가들인 부여, 송화, 진한, 변한 등이 등장하게 됩니다.\n\n2. **삼국시대**: 1세기경부터 7세기 중반까지 고구려, 백제, 신라의 삼국이 한반도를 주종으로 다스렸습니다. 이들은 각각 독자적인 문화와 정치 체제를 갖추었으며, 7세기에는 신라가 대부분의 한반도를 통합하게 됩니다.\n\n3. **통일신라와 발해**: 676년 신라가 삼국을 통일한 후, '통일신라' 시대가 시작됩니다. 이 시기에는 불교문화가 발달했고, 이후 발해라는 새로운 국가가 북쪽에서 등장해 한반도 북부와 만주 지역을 아우렀습니다.\n\n4. **고려시대**: 918년 왕건이 고려를 건국하여 1392년까지 지속되었습니다. 고려는 불교 중심의 문화가 발달하였고, 청자와 같은 뛰어난 도자기가 제작되었습니다. 몽골의 침입과 같은 외부의 위협도 견뎌냈습니다.\n\n5. **조선시대**: 1392년에서 1910년까지 존재한 조선은 유교를 국가 이념으로 삼았습니다. 이 시기에 한글이 창제되었고, 과학, 기술, 예술 등의 분야에서 발전을 이루었습니다. 그러나 19세기 말부터 일본 제국의 압박을 받기 시작했습니다.\n\n6. **일제 강점기**: 1910년부터 1945년까지 한국은 일본의 식민지로 지배받았습니다. 이 시기에 많은 문화와 역사적 자산이 왜곡되고 파괴되었습니다.\n\n7. **남북 분단과 현대사**: 제2차 세계대전 이후 1945년 한반도는 북위 38도선을 기준으로 남북으로 나뉘었습니다. 1950년에는 한국전쟁이 발발하였고, 전쟁 후 1953년 휴전협정이 체결되었습니다. 현재 남한은 민주주의 체제를 갖춘 반면, 북한은 공산주의 체제를 유지하고 있습니다.\n\n8. **현대 한국**: 남한은 경제 발전과 민주화를 이루며 글로벌 강국

In [29]:
# 각 체인 구성: 역사와 경제에 대한 요청을 처리
chain1 = prompt1 | model | parser  # 역사 체인
chain2 = prompt2 | model | parser  # 경제 체인

# RunnableParallel을 사용하여 두 개의 체인을 병렬로 처리
parallel = RunnableParallel(history=chain1, economy=chain2)

# '한국'에 대한 정보를 병렬로 요청
response = parallel.invoke({'country': '한국'})  # 한국의 역사와 경제 정보 요청

# 결과 출력
print(response)  # 한국의 역사와 경제 정보 출력

{'history': "한국의 역사는 매우 풍부하고 다양한 문화와 사건들이 얽혀 있습니다. 아래는 한국 역사에 대한 간략한 개요입니다.\n\n1. **고대**:\n   - **삼국시대 (1세기 - 7세기)**: 고구려, 백제, 신라의 세 나라가 한반도와 주변 지역에서 세력을 다툴 때 시기입니다. 이 시기에 불교가 전래되고, 각국의 문화가 발전하였습니다.\n   - **고려시대 (918 - 1392)**: 태조 왕건이 고려를 세우고, 후에 몽골의 침입을 받으면서도 문화와 불교가 발전하였습니다.\n\n2. **조선시대 (1392 - 1910)**:\n   - 이성계가 조선을 세우고, 유교가 국가의 이념으로 자리잡습니다. 이 시기에는 많은 문화적, 과학적 발전이 이루어졌고, 한글이 창제되었습니다.\n   - 조선후기에는 서구의 영향을 받으며 변화가 시작되었습니다.\n\n3. **일제강점기 (1910 - 1945)**:\n   - 한국은 일본에 의해 식민지로 지배받으며, 많은 고통을 겪었습니다. 이 시기에 독립을 위한 다양한 운동이 일어났습니다.\n\n4. **광복 및 분단 (1945 - 1950)**:\n   - 제2차 세계대전이 끝나고 한국이 일본으로부터 해방되지만, 냉전의 영향으로 남북이 각각 소련과 미국의 영향 아래에서 분단되었습니다.\n\n5. **한국전쟁 (1950 - 1953)**:\n   - 북한과 남한 간의 전쟁이 발발하였고, 결과적으로 휴전선이 형성되어 남북한이 분단 상태로 남게 됩니다.\n\n6. **현대 (1953 - 현재)**:\n   - 1960년대부터 경제 발전이 급속하게 진행되었고, '한강의 기적'으로 불리는 경제 성장을 이루었습니다.\n   - 1987년 민주화 운동을 통해 민주 정부가 수립되었고, 현재는 글로벌 경제체제의 중요한 일원이 되었습니다.\n\n한국 역사는 이러한 주요 사건들 외에도 많은 변화와 발전이 있었으며, 전통과 현대가 조화를 이루는 독특한 문화를 형성해왔습니다.",
 'economy': '한국의 경제 구조는 다음과 같

In [30]:
## example
# 한 국가의 수도, GDP, 인구, 위도, 경도 정보를 받으려고 한다
# 5개의 chain을 만들어 각각의 정보를 수집할 수 있도록 한다.
# 국가는 최소 10개 이상의 국가를 입력하며, 최대한 빠른 시간 내에 처리할 수 있도록 한다.
# 다만, 서버 상황을 고려하여 최대 동시요청 횟수는 5회로 제한한다.

# 각 국가에 대한 다양한 정보를 요청하기 위한 템플릿 정의
template1 = '{country}의 수도가 어딘지 알려줘'
template2 = '{country}의 GDP가 어떻게 되는지 알려줘'
template3 = '{country}의 인구가 어떻게 되는지 알려줘'
template4 = '{country}의 위도를 알려줘'
template5 = '{country}의 경도를 알려줘'

# 각 템플릿에 대해 PromptTemplate 객체 생성
prompt1 = PromptTemplate.from_template(template1)
prompt2 = PromptTemplate.from_template(template2)
prompt3 = PromptTemplate.from_template(template3)
prompt4 = PromptTemplate.from_template(template4)
prompt5 = PromptTemplate.from_template(template5)

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=0,              # 정확한 응답을 위해 온도를 낮게 설정
)

# 출력 파서를 설정: 모델의 출력을 문자열 형태로 처리하는 파서
parser = StrOutputParser()

# 각 체인 구성: 국가에 대한 다양한 요청을 처리
chain1 = prompt1 | model | parser  # 수도 요청 체인
chain2 = prompt2 | model | parser  # GDP 요청 체인
chain3 = prompt3 | model | parser  # 인구 요청 체인
chain4 = prompt4 | model | parser  # 위도 요청 체인
chain5 = prompt5 | model | parser  # 경도 요청 체인

# RunnableParallel을 사용하여 여러 요청을 병렬로 처리
parallel = RunnableParallel(
    capital=chain1,
    gdp=chain2,
    population=chain3,
    latitude=chain4,
    longitude=chain5,
)

# 여러 국가에 대한 정보를 비동기 배치 요청
response = await parallel.abatch([
    {'country': '한국'},
    {'country': '일본'},
    {'country': '중국'},
    {'country': '방글라데시'},
    {'country': '베트남'},
    {'country': '이란'},
    {'country': '인도'},
    {'country': '미국'},
    {'country': '싱가포르'},
    {'country': '요르단'},
],
config={
    'max_concurrency': 5,  # 최대 동시 요청 수를 5로 설정
}
)

# 결과 출력
print(response)  # 요청에 대한 응답 출력


[{'capital': '한국의 수도는 서울입니다.',
  'gdp': '2023년 기준으로 한국의 GDP(국내총생산)는 약 1.8조 달러(약 2,000조 원) 정도로 추정됩니다. 한국은 세계에서 10위권 내의 경제 규모를 가진 국가로, 제조업, 서비스업, IT 산업 등이 주요 산업으로 자리잡고 있습니다. 정확한 수치는 매년 변동이 있을 수 있으므로, 최신 정보를 확인하려면 한국은행이나 통계청의 공식 발표를 참고하는 것이 좋습니다.',
  'population': '2023년 기준으로 한국의 인구는 약 5,100만 명 정도로 추정됩니다. 한국은 고령화 사회로, 출생률이 낮아지고 있으며, 이로 인해 인구 감소 문제도 대두되고 있습니다. 정확한 인구 수치는 통계청이나 관련 기관의 최신 자료를 참고하는 것이 좋습니다. 추가적인 정보가 필요하시면 말씀해 주세요!',
  'latitude': '한국의 위도는 대략 33도에서 43도 사이에 위치하고 있습니다. 구체적으로, 서울의 위도는 약 37.5665도 북위입니다. 한국은 남쪽의 제주도에서 북쪽의 북한과의 경계까지 다양한 위도를 가지고 있습니다.',
  'longitude': '한국의 경도는 대략 124도 동경에서 132도 동경 사이에 위치하고 있습니다. 한국의 주요 도시들, 예를 들어 서울은 약 126도 58분 동경에 위치하고 있습니다. 경도는 지구의 동서 방향 위치를 나타내는 좌표로, 한국은 동경 기준으로 동쪽에 위치하고 있습니다.'},
 {'capital': '일본의 수도는 도쿄(東京)입니다. 도쿄는 일본의 정치, 경제, 문화의 중심지로, 많은 인구와 다양한 시설이 있는 대도시입니다.',
  'gdp': '2023년 기준으로 일본의 GDP(국내총생산)는 약 4조 달러 정도로 추정됩니다. 일본은 세계에서 세 번째로 큰 경제 규모를 가진 나라로, 제조업, 서비스업, 기술 산업 등이 주요 산업입니다. 정확한 수치는 매년 변동할 수 있으므로, 최신 정보를 확인하려면 공식 통계나 경제 관련 뉴스 자료를 참고하는 것이 좋습니

## Cache

### Memory Cache

캐시를 메모리에 저장. <br>
애플리케이션이 실행되는 동안만 유효. <br>
-> 빠른 응답이 필요하지만, 지속적인 저장이 필요하지 않은 경우 적합.

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> set_llm_cache(InMemoryCache())
> 
> llm = ChatOpenAI(
>     temperature=0,
>     model_name="gpt-4o-mini",
> )
> 
> chain.invoke()
> ...
> ```

In [ ]:
# %%timeit 매직 명령어: 아래 코드를 반복 실행하여 평균 실행 시간을 측정
%%timeit -n 5 -r 5

# ChatOpenAI 모델 설정: 사용할 모델과 온도 설정
llm = ChatOpenAI(
    model='gpt-4o-mini',  # 사용할 모델 이름
    temperature=0,        # 정확한 응답을 위해 온도를 낮게 설정
)

# 질문 정의: 제주도의 관광 명소에 대한 질문
question = '제주도의 관광 명소에 대해서 알려줘'

# 모델에 질문을 전달하여 응답을 받음
response = llm.invoke(question)  # 질문에 대한 응답 호출

6.63 s ± 659 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [36]:
# InMemoryCache를 사용하여 LLM 응답을 캐시하는 설정
set_llm_cache(InMemoryCache())


In [37]:
# %%timeit 매직 명령어: 아래의 코드를 반복 실행하여 평균 실행 시간을 측정
%%timeit -n 5 -r 5

# ChatOpenAI 모델 설정
llm = ChatOpenAI(
    model='gpt-4o-mini',  # 사용할 모델 이름
    temperature=0,        # 응답의 일관성을 위해 온도를 0으로 설정
)

# 질문 정의: 제주도의 관광 명소에 대한 질문
question = '제주도의 관광 명소에 대해서 알려줘'

# 모델에 질문을 전달하여 응답을 받음
response = llm.invoke(question)  # 질문에 대한 응답을 호출

The slowest run took 5.62 times longer than the fastest. This could mean that an intermediate result is being cached.
557 ms ± 534 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


### SQLAlchemy Cache

SQLAlchemy Cache는 다양한 데이터베이스를 지원하는 SQLAlchemy를 사용하여 캐시 관리. <br>
 -> MySQL, PostgreSQL 등 다양한 데이터베이스에서 캐시 사용 가능.
 
<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> engine = create_engine("sqlite:///sqlalchemy_cache.db")
> llm_cache = SQLAlchemyCache(engine=engine)
> set_llm_cache(llm_cache)
> 
> llm = ChatOpenAI(
>     temperature=0,
>     model_name="gpt-4o-mini",
> )
> 
> chain.invoke()
> ...
> ```

## Prompt

프롬프트 단계는 검색된 문서들을 바탕으로 언어 모델이 사용할 질문이나 명령을 생성하는 과정. <br>
검색된 정보를 이용하여 사용자의 질문에 가장 잘 답할 수 있는 응답을 생성하기 위해 필수적인 단계. <br>


### Prompt Template

<font style='font-size:18px'> Basic </font>

> ```python
> # template define
> template = '{country}의 인구는 몇명인가요?'
> 
> # from_template을 통하여 PromptTemplate 객체 생성
> prompt_template = PromptTemplate.from_template(template)
> prompt_template
> 
> # prompt 생성
> prompt = prompt_template.format(country='대한민국')
> 
> 
> # PromptTemplate으로 생성
> template = '{country1}과 {country2}의 인구는 각각 얼마니?'
> 
> # prompt 생성
> prompt = PromptTemplate(
>     template=template,
>     input_variables=['country1'],
>     partial_variables={
>         'country2': '일본'    #   country2에 사용할 default 값
>     },
> )
> 
> prompt.format(country1='한국')
> 
> # country2의 default값 변경
> prompt = prompt.partial(country2='미국')  # 변경된 값을 사용하려면 반드시 재할당
> ```

<br>

<font style='font-size:18px'> Chat </font>

대화목록을 프롬프트에 입력 시 사용. <br>
메시지는 튜플로 구성하며, (role, message)의 리스트로 생성.

<br>

> ```python
> prompt = ChatPromptTemplate.from_messages(
>     [
>         ('system', 'You are a helpful assistant. 당신은 {object}을(를) 잘 수행해야 한다.'),
>         ('human', 'can you explain {financial_statement}?'),
>     ]
> )
> 
> message = chat_template.format_messages(
>     object='재무제표 분석', financial_statement=financial_statement
> )
> chain = prompt | llm | StrOutputParser()
> ```

<br>

<font style='font-size:18px'> MessagePlaceHolder </font>

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 메시지 목록을 삽입하려는 경우 사용. <br>

> ```python
> prompt = ChatPromptTemplate.from_messages(
>     [
>         ('system', 'You are a helpful assistant. 당신은 {object}을(를) 잘 수행해야 한다.'),
>          MessagesPlaceholder(variable_name='example'),
>         ('human', 'can you explain {financial_statement}?'),
>     ]
> )
>
> new_prompt = chat_prompt.format(
>     conversation=[
>         ('ai', '아래는 재무제표의 요약 예이다. 자산: , 자본: , 부채: '),
>     ],
> )
>
> chain = new_prompt | llm | StrOutputParser()
> ```

In [44]:
# 템플릿 정의: 특정 연도, 국가, 수도에 대한 인구 수를 질문하는 형식
template = '{year}년도에 {country}의 {capital}에 몇 명의 인구가 거주해?'

# PromptTemplate 객체 생성: 입력 변수와 부분 변수 설정
prompt = PromptTemplate(
    template=template,
    input_variables=['country', 'capital'],  # 필요한 입력 변수
    partial_variables={                       # 기본값으로 설정할 부분 변수
        'year': '2020'
    }
)

# year 변수를 2022로 변경하여 새로운 부분 템플릿 생성
prompt = prompt.partial(year='2022')

# 생성된 템플릿 출력
prompt

PromptTemplate(input_variables=['capital', 'country'], input_types={}, partial_variables={'year': '2022'}, template='{year}년도에 {country}의 {capital}에 몇 명의 인구가 거주해?')

In [50]:
# chat
# ChatPromptTemplate을 사용하여 대화형 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ('system', 'you are a helpful assistant. your goal is to explain {object}'),  # 시스템 역할 정의
    ('human', 'can you explain {subject}?'),  # 인간 사용자 질문 정의
])

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,        # 응답의 창의성을 높이기 위해 높은 온도 설정
)

# 출력 파서 설정: 모델의 출력을 처리할 파서
parser = StrOutputParser()

# 프롬프트, 모델, 파서를 연결하여 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 특정 객체와 주제에 대한 설명 요청
response = chain.invoke({'object': 'financial statement', 'subject': 'accounting equation'})

# 결과 출력
print(response)  # 'financial statement'와 관련된 'accounting equation' 설명 출력

Certainly! The accounting equation is a fundamental principle of accounting that represents the relationship between a company's assets, liabilities, and equity. The equation is expressed as:

**Assets = Liabilities + Equity**

Here's a breakdown of each component:

1. **Assets**: These are resources owned by a business that have economic value and can provide future benefits. Assets can include cash, accounts receivable, inventory, property, equipment, and investments.

2. **Liabilities**: These are obligations or debts that a business owes to external parties. Liabilities can result from borrowing money, purchasing goods or services on credit, or any other financial commitments. Examples include loans, accounts payable, and mortgages.

3. **Equity**: This represents the owner's claim on the assets of the business after all liabilities have been deducted. Equity can come from two main sources:
   - **Invested Capital**: Money that owners or shareholders have invested in the business.


In [52]:
# 프롬프트 메시지를 실제 값으로 포맷하여 채팅 템플릿 생성
chat_template = prompt.format_messages(
    object='financial statement',  # 'object' 변수에 'financial statement' 할당
    subject='accounting equation'   # 'subject' 변수에 'accounting equation' 할당
)

# 생성된 채팅 템플릿 출력
chat_template

[SystemMessage(content='you are a helpful assistant. your goal is to explain financial statement', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='can you explain accounting equation?', additional_kwargs={}, response_metadata={})]

In [56]:
## chat
# ChatPromptTemplate을 사용하여 대화형 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ('system', 'you are a helpful assistant. your goal is to explain {object}'),  # 시스템 역할 정의
    MessagesPlaceholder(variable_name='chat'),  # 이전 대화 메시지를 위한 자리 표시자
    ('human', 'can you explain {subject}?'),  # 인간 사용자 질문 정의
])

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,        # 응답의 창의성을 높이기 위해 높은 온도 설정
)

# 출력 파서 설정: 모델의 출력을 처리할 파서
parser = StrOutputParser()

# 프롬프트, 모델, 파서를 연결하여 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 객체와 주제, 이전 대화 맥락에 대한 설명 요청
response = chain.invoke({
    'object': 'financial statement',
    'subject': 'accounting equation',
    'chat': [
        ('human', '재무제표가 주어지면 자산의 관점에서 자세하게 설명.'),
        ('ai', '자산은 유동자산 비유동자산으로 구분.')
    ]
})

# 결과 출력
print(response)  # AI의 응답 출력

The accounting equation is the foundational principle of double-entry bookkeeping, and it represents the relationship between a company’s assets, liabilities, and equity. The equation is expressed as:

**Assets = Liabilities + Equity**

Here’s a breakdown of each component:

1. **Assets**: These are resources owned by a company that have economic value and can provide future benefits. Assets can be classified into two main categories:
   - **Current Assets**: Assets that are expected to be converted into cash or used within one year, such as cash, accounts receivable, inventory, and short-term investments.
   - **Non-Current Assets** (or Long-Term Assets): Assets that are not expected to be converted into cash within one year, such as property, plant and equipment, long-term investments, and intangible assets (like patents and trademarks).

2. **Liabilities**: These are obligations or debts that a company owes to external parties. Liabilities can also be classified into:
   - **Current

In [ ]:
# 프롬프트 템플릿을 실제 값으로 포맷하여 채팅 메시지를 생성
formatted_prompt = prompt.format(
    object='financial statement',  # 'object' 변수에 'financial statement' 할당
    subject='accounting equation',  # 'subject' 변수에 'accounting equation' 할당
    chat=[  # 이전 대화 맥락을 정의
        ('human', '재무제표가 주어지면 자산의 관점에서 자세하게 설명.'),
        ('ai', '자산은 유동자산 비유동자산으로 구분.')
    ]
)

# 생성된 포맷된 프롬프트 출력
formatted_prompt

In [57]:
## day48 김치프리미엄
# ChatPromptTemplate을 사용하여 대화형 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ('system', '''주어지는 작은 따옴표(\') 또는 큰 따옴표(") 세 개로 쌓인 텍스트 내에서 유저의 질문에 대한 답을 해줘.
                  작은 따옴표 세 개나 큰 따옴표 세 개로 쌓인 텍스트 내에서 답을 찾을 수 없는 경우,
                  해당 질문에 대한 답을 주어진 텍스트에서 찾을 수 없다고 안내해줘.'''),  # 시스템 메시지
    MessagesPlaceholder(variable_name='references'),  # 참조할 텍스트 자리 표시자
    ('human', 'Question: {year}년에 김치프리미엄이 50% 근처였는가?'),  # 질문 정의
])

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model='gpt-4o-mini',  # 사용할 모델 이름
    temperature=0,        # 응답의 일관성을 위해 온도를 0으로 설정
)

# 출력 파서 설정: 모델의 출력을 처리할 파서
parser = StrOutputParser()

# 프롬프트, 모델, 파서를 연결하여 체인 구성
chain = prompt | model | parser

article = '''
김치프리미엄 사라져 가격 역전
국내 업비트서 0.8% 낮게 거래
美 비트코인ETF로 유동성 증가
韓선 기관 투자 막혀 거래량 정체
 
최근 국내에서 비트코인이 해외보다 오히려 낮은 가격에 거래되는 것으로 나타났다. 올해 초까지만 해도 국내 가상화폐 시장에 개인투자자 자금이 대거 몰려 해외에 비해 시세가 높게 형성되는 ‘김치 프리미엄’ 현상이 나타났지만, 지금은 반대 상황이 됐다. 상장지수펀드(ETF)가 승인되며 기관투자가 진입이 원활해진 미국 등과 달리 국내 가상화폐 시장은 개인 투자 수요에만 의존하면서 증시에 이어 가상자산에서도 일종의 ‘코리아 디스카운트’가 발생하고 있다는 분석이 나온다.

● 국내-해외 비트코인 가격 ‘역전’

17일 가상자산 시황 비교 플랫폼 ‘크라이프라이스’에 따르면 이날 오후 3시 기준 글로벌 가상화폐 거래소 바이낸스에서 비트코인은 1개당 약 6만7415달러(약 9226만 원)에 거래됐다. 같은 시각 국내 가상화폐 거래소인 업비트에서 비트코인은 약 9150만 원에 거래 중이었다. 국내에서 거래되는 비트코인이 약 0.8% 싸게 사고 팔리는 셈이다.

국내 가상자산 시장은 2017년 이후 과열 기미를 보이면서 비트코인의 원화 거래 가격이 달러화 가격보다 10∼20% 비싼 상태가 이어져 왔다. 이에 웃돈(프리미엄)을 주고 사야 한다는 의미로 김치 프리미엄이라는 용어가 사용됐다. 이런 현상은 국내 비트코인 거래량이 급증했던 올해 초까지 이어져 왔다.

그러나 최근 글로벌 가상자산 시장에 자금이 유입되며 비트코인 가격이 강세를 보이는 반면, 국내에선 거래량이 늘지 않으면서 가격 역전이 발생했다. 지난달 미국 연방준비제도(Fed·연준)가 기준금리를 한 번에 0.5%포인트 낮추는 ‘빅 컷’을 단행해 위험 자산으로 돈이 돌아온 데 이어 가상자산에 친화적인 도널드 트럼프 미국 공화당 대통령 후보의 지지율이 오르면서 생긴 현상이다.

● 비트코인 규제로 국내 시장은 차분

해외에서 비트코인 가격이 상승하는 또 다른 요인으로는 올해 1월 미국에서 상장된 비트코인 현물 ETF가 꼽힌다. ETF를 통해 기관 자금이 유입되며 유동성이 늘어난 데 더해 금리 인하 등 호재가 맞물리며 상승 폭이 커졌다는 것이다. 김민승 코빗 리서치센터장은 “올해 비트코인 가격 상승의 주동력은 미국 증시에 상장된 현물 ETF를 통해 들어온 자금”이라고 분석했다. 반면 국내의 경우 가상자산 현물 ETF 상장이 막혀 현실적으로 주요 기관의 가상자산 투자는 전면 차단돼 있다. 비트코인 거래가 개인에 의존하다 보니 거래량이 크게 증가하지 않는다는 것이다. 여기에 더해 기존의 국내 비트코인 투자자들이 미국 주식 등 해외 자산 투자로 눈을 돌리는 것도 국내 가상자산 가격 약세를 부채질하고 있다.

김치 프리미엄이 사라진 것을 긍정적으로 바라보는 시각도 있다. 일부 세력에 의한 시세 조종 등 불공정 거래가 줄어든 결과라는 것이다. 천창민 서울과학기술대 글로벌테크노경영학과 교수는 “가상자산 이용자 보호법이 도입되면서 시장이 건전해진 영향”이라며 “과거엔 비트코인을 대거 보유한 ‘대형 고래’들이 가격을 조작할 수 있었는데, 강력한 규제가 도입되면서 영향력이 줄어든 것”이라고 설명했다.
'''

# 체인을 사용하여 질문과 참조 텍스트를 모델에 전달하고 응답을 출력
print(
    chain.invoke({
        'year': '2017',  # 질문에 사용할 연도 설정
        'references': [  # 참조할 텍스트 정의
            ('human', f'"""{article}"""'),  # 'article' 변수를 포함한 텍스트
        ]
    })
)

주어진 텍스트에서 김치프리미엄이 50% 근처였다는 언급은 없습니다. 해당 질문에 대한 답을 찾을 수 없습니다.


### FewShotPromptTemplate

모델에게 주어진 예시들의 패턴을 보여주거나 특정한 반응 방식을 답변받기 위해 사용. <br>
-> 모델이 의도된 작업을 충분히 이해하기 어려운 복잡하거나 미묘한 작업에 특히 유용. <br>

<br>

> ```python
> examples = [
>     {
>         'question': '스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?',
>         'answer': '''이 질문에 추가 질문이 필요한가요: 예.
>                     추가 질문: 스티브 잡스는 몇 살에 사망했나요?
>                     중간 답변: 스티브 잡스는 56세에 사망했습니다.
>                     추가 질문: 아인슈타인은 몇 살에 사망했나요?
>                     중간 답변: 아인슈타인은 76세에 사망했습니다.
>                     최종 답변은: 아인슈타인
>                     ''',
>     },
>     {
>         'question': '네이버의 창립자는 언제 태어났나요?',
>         'answer': '''이 질문에 추가 질문이 필요한가요: 예.
>                     추가 질문: 네이버의 창립자는 누구인가요?
>                     중간 답변: 네이버는 이해진에 의해 창립되었습니다.
>                     추가 질문: 이해진은 언제 태어났나요?
>                     중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
>                     최종 답변은: 1967년 6월 22일
>                     ''',
>     },
>     {
>         'question': '율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?',
>         'answer': '''이 질문에 추가 질문이 필요한가요: 예.
>                     추가 질문: 율곡 이이의 어머니는 누구인가요?
>                     중간 답변: 율곡 이이의 어머니는 신사임당입니다.
>                     추가 질문: 신사임당은 언제 태어났나요?
>                     중간 답변: 신사임당은 1504년에 태어났습니다.
>                     추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
>                     중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
>                     최종 답변은: 연산군
>                     ''',
>     },
>     {
>         'question': '올드보이와 기생충의 감독이 같은 나라 출신인가요?',
>         'answer': '''이 질문에 추가 질문이 필요한가요: 예.
>                     추가 질문: 올드보이의 감독은 누구인가요?
>                     중간 답변: 올드보이의 감독은 박찬욱입니다.
>                     추가 질문: 박찬욱은 어느 나라 출신인가요?
>                     중간 답변: 박찬욱은 대한민국 출신입니다.
>                     추가 질문: 기생충의 감독은 누구인가요?
>                     중간 답변: 기생충의 감독은 봉준호입니다.
>                     추가 질문: 봉준호는 어느 나라 출신인가요?
>                     중간 답변: 봉준호는 대한민국 출신입니다.
>                     최종 답변은: 예
>                     ''',
>     },
> ]
> 
> example_prompt = PromptTemplate.from_template(
>     'Question:\n{question}\nAnswer:\n{answer}'
> )
> 
> prompt = FewShotPromptTemplate(
>     examples=examples,
>     example_prompt=example_prompt,
>     suffix='Question:\n{question}\nAnswer:',
>     input_variables=['question'],
> )
> 
> chain = prompt | llm | StrOutputParser()
> 
> answer = chain.stream(
>     {'question': '1920년대 미국과 영국의 경제상황은 비슷한가?'}
> )
> ```

In [60]:
# 질문 템플릿 정의: '{year}년대 미국과 영국의 경제상황은 비슷한가?'
template = '{year}년대 미국과 영국의 경제상황은 비슷한가?'
prompt = PromptTemplate.from_template(template)  # 프롬프트 템플릿 생성

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=0,              # 응답의 일관성을 위해 온도를 0으로 설정
)

# 출력 파서 설정: 모델의 출력을 처리할 파서
parser = StrOutputParser()

# 프롬프트, 모델, 파서를 연결하여 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 1920년에 대한 질문을 모델에 전달하고 응답 출력
print(chain.invoke({'year': '1920'}))

1920년대 미국과 영국의 경제 상황은 몇 가지 유사점과 차이점이 있었습니다. 

**유사점:**
1. **전후 경제 회복**: 두 나라 모두 제1차 세계대전 이후 경제 회복을 경험했습니다. 전쟁 동안의 산업 생산 증가와 수출 확대가 경제 성장에 기여했습니다.
2. **소비 문화의 확산**: 1920년대는 소비주의가 확산된 시기로, 미국과 영국 모두에서 대중 소비가 증가했습니다. 자동차, 가전제품 등의 소비가 활발해졌습니다.

**차이점:**
1. **경제 성장률**: 미국은 1920년대에 강력한 경제 성장을 경험하며 '광란의 20년대'로 알려진 시기를 맞이했습니다. 반면, 영국은 전후 경제 회복이 더디고, 산업 구조의 변화에 어려움을 겪었습니다.
2. **실업률과 사회적 문제**: 영국은 전후 실업률이 높았고, 특히 전통적인 산업(예: 석탄, 섬유)에서의 일자리가 줄어들면서 사회적 불안이 증가했습니다. 미국은 상대적으로 낮은 실업률을 유지하며 경제가 활황을 누렸습니다.
3. **금융 시스템**: 미국은 1920년대에 금융 시장이 급격히 성장하며 주식 시장이 활성화되었습니다. 반면, 영국은 전통적인 금융 시스템에 의존하고 있었고, 주식 시장의 성장 속도가 느렸습니다.

결론적으로, 1920년대 미국과 영국의 경제 상황은 전반적으로 비슷한 점도 있었지만, 성장 속도와 사회적 문제에서 뚜렷한 차이를 보였습니다.


In [61]:
examples = [
    {
        'question': '스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?',
        'answer': '''이 질문에 추가 질문이 필요한가요: 예.
                    추가 질문: 스티브 잡스는 몇 살에 사망했나요?
                    중간 답변: 스티브 잡스는 56세에 사망했습니다.
                    추가 질문: 아인슈타인은 몇 살에 사망했나요?
                    중간 답변: 아인슈타인은 76세에 사망했습니다.
                    최종 답변은: 아인슈타인
                    ''',
    },
    {
        'question': '네이버의 창립자는 언제 태어났나요?',
        'answer': '''이 질문에 추가 질문이 필요한가요: 예.
                    추가 질문: 네이버의 창립자는 누구인가요?
                    중간 답변: 네이버는 이해진에 의해 창립되었습니다.
                    추가 질문: 이해진은 언제 태어났나요?
                    중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
                    최종 답변은: 1967년 6월 22일
                    ''',
    },
    {
        'question': '율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?',
        'answer': '''이 질문에 추가 질문이 필요한가요: 예.
                    추가 질문: 율곡 이이의 어머니는 누구인가요?
                    중간 답변: 율곡 이이의 어머니는 신사임당입니다.
                    추가 질문: 신사임당은 언제 태어났나요?
                    중간 답변: 신사임당은 1504년에 태어났습니다.
                    추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
                    중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
                    최종 답변은: 연산군
                    ''',
    },
    {
        'question': '올드보이와 기생충의 감독이 같은 나라 출신인가요?',
        'answer': '''이 질문에 추가 질문이 필요한가요: 예.
                    추가 질문: 올드보이의 감독은 누구인가요?
                    중간 답변: 올드보이의 감독은 박찬욱입니다.
                    추가 질문: 박찬욱은 어느 나라 출신인가요?
                    중간 답변: 박찬욱은 대한민국 출신입니다.
                    추가 질문: 기생충의 감독은 누구인가요?
                    중간 답변: 기생충의 감독은 봉준호입니다.
                    추가 질문: 봉준호는 어느 나라 출신인가요?
                    중간 답변: 봉준호는 대한민국 출신입니다.
                    최종 답변은: 예
                    ''',
    },
]

In [66]:
# 질문과 답변을 포함하는 템플릿 정의
template = 'Question:\n{question}\nAnswer:\n{answer}'
example_prompt = PromptTemplate.from_template(template)  # 템플릿을 기반으로 프롬프트 생성

# FewShotPromptTemplate 설정: 사전 정의된 예시와 함께 사용할 프롬프트 정의
prompt = FewShotPromptTemplate(
    examples=examples,  # 사전 정의된 예시 목록
    example_prompt=example_prompt,  # 예시 프롬프트
    suffix='Question:\n{question}\nAnswer:',  # 추가 질문 프롬프트
    input_variables=['question']  # 입력 변수로 질문 정의
)

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=0,              # 응답의 일관성을 위해 온도를 0으로 설정
)

# 출력 파서 설정: 모델의 출력을 처리할 파서
parser = StrOutputParser()

# 프롬프트, 모델, 파서를 연결하여 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 주어진 질문에 대한 응답을 모델에 전달하고 출력
print(chain.invoke({'question': '1920년대 미국과 영국의 경제상황은 비슷한가?'}))

이 질문에 추가 질문이 필요한가요: 예.  
                    추가 질문: 1920년대 미국의 경제상황은 어땠나요?  
                    중간 답변: 1920년대 미국은 경제 호황을 경험했으며, '재즈 시대'로 불리는 시기로 소비와 산업이 급격히 성장했습니다.  
                    추가 질문: 1920년대 영국의 경제상황은 어땠나요?  
                    중간 답변: 1920년대 영국은 제1차 세계대전 이후 경제 회복에 어려움을 겪었고, 실업률이 높아지며 경제가 침체되었습니다.  
                    최종 답변은: 아니오


In [72]:
# 질문과 답변 예시를 정의하는 리스트
examples = [
    {
        'question': '8, 11, 10, 14, 14, 19',  # 수열 질문
        'answer': 20  # 다음 숫자
    },
    {
        'question': '11, 14, 13, 17, 17, 22',
        'answer': 23
    },
    {
        'question': '1, 4, 3, 7, 7, 12',
        'answer': 13
    },
]

# 질문과 답변 형식을 정의하는 템플릿
template = 'Question:\n{question}\nAnswer:{answer}'
# 템플릿을 기반으로 프롬프트 생성
example_prompt = PromptTemplate.from_template(template)

# FewShotPromptTemplate 설정: 예시를 기반으로 새로운 질문을 처리
prompt = FewShotPromptTemplate(
    examples=examples,  # 사전 정의된 예시 목록
    example_prompt=example_prompt,  # 예시 프롬프트
    suffix='Question:\n{question}\nAnswer:',  # 추가 질문 프롬프트
    input_variables=['question']  # 입력 변수로 질문 정의
)

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,              # 응답의 다양성을 조절하는 온도
)

# 출력 파서를 설정하여 모델의 출력을 처리
parser = StrOutputParser()

# 프롬프트, 모델, 파서를 연결하여 하나의 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 주어진 질문에 대한 응답을 모델에 전달하고 출력
print(chain.invoke({'question': '-5, -1, -3, 2, 1, 7'}))  # 새로운 수열에 대한 예측 요청

To solve this, let's first identify any patterns in the given sequences and their corresponding answers.

1. **First sequence:** 
   - Numbers: 8, 11, 10, 14, 14, 19
   - Answer: 20 

   The average of the first five numbers (8 + 11 + 10 + 14 + 14) is 11.4, which doesn't help. Another way to approach: The next number seems to go up by 1 from the highest number (19) to get to 20. 

2. **Second sequence:** 
   - Numbers: 11, 14, 13, 17, 17, 22
   - Answer: 23 

   Here, again looking at the last number. The maximum number is 22, and the answer goes to the next integer, which is 23.

3. **Third sequence:** 
   - Numbers: 1, 4, 3, 7, 7, 12
   - Answer: 13 

   Similar reasoning: The last number is 12, and the answer is again the next integer, which is 13. 

Following this pattern, we look at the fourth sequence:

**Sequence:**
- Numbers: -5, -1, -3, 2, 1, 7

The maximum here is 7, so the next integer after 7 is 8.

Thus, the answer to the last question should be:

**Answer: 8**


### Langchain Hub

다양한 prompt를 hub로부터 다운받거나 나의 prompt를 업로드

<br>

<font style="font-size:20px"> Hub로부터 Prompt 받아오기 </font>

> ```python
> prompt = hub.pull('rlm/rag-prompt')
> ```

<br>

<font style="font-size:20px"> Hub에 Prompt 올리기 </font>

> ```python
> prompt = hub.push('<user_name>/<prompt_name>', prompt)
> ```



In [67]:
# 특정 프롬프트를 불러오기 위해 hub에서 'rlm/rag-prompt'를 가져옵니다.
prompt = hub.pull('rlm/rag-prompt')

C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [68]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## Output Parsers

LLM의 출력을 구조화된 형태로 변환하는 컴포넌트

### PydanticOutputParser

언어 모델의 출력을 구조화된 정보로 변환 하는 데 도움을 주는 클래스. <br>
단순 텍스트 형태의 응답 대신, 사용자가 원하는 형식으로 답변 제공 가능.

아래의 두 핵심 method로 구현 <br>
- get_format_instruction: 출력해야 할 형식을 정의하는 instruction 전달
- parse: 특정 구조로 변환

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> prompt = PromptTemplate.from_template(
> """
> You are a helpful assistant. 아래 질문에 맞게 한국어로 ㄷ바하라.
> 
> question:
> {question}
> 
> 재무제표:
> {financial_statement}
> 
> format:
> {format}
> """
> )
> 
> class FinancialStatementSummary(BaseModel):
>     asset: str = Field(description='자산')
>     capital: str = Field(description='자본')
>     liability: str = Field(description='부채')
> 
> parser = PydanticOutputParser(pydantic_object=FinancialStatementSummary)
> prompt = prompt.partial(format=parser.get_format_instructions())
> 
> 
> model = 
> 
> chain = prompt | llm | parser
> 
> response = chain.invoke(
>     {
>         "financial_statement": financial_statement,
>         "question": "재무제표에서 핵심 사항을 요약하라.",
>     }
> )
> ```

In [73]:
financial_statement = '''
2-1. 연결 재무상태표

연결 재무상태표

제 55 기          2023.12.31 현재

제 54 기          2022.12.31 현재

제 53 기          2021.12.31 현재

(단위 : 백만원)

　	
제 55 기

제 54 기

제 53 기

자산

　	　	　
　유동자산

195,936,557

218,470,581

218,163,185

　　현금및현금성자산 (주4,28)

69,080,893

49,680,710

39,031,415

　　단기금융상품 (주4,28)

22,690,924

65,102,886

81,708,986

　　단기상각후원가금융자산 (주4,28)

608,281

414,610

3,369,034

　　단기당기손익-공정가치금융자산 (주4,6,28)

27,112

29,080

40,757

　　매출채권 (주4,5,7,28)

36,647,393

35,721,563

40,713,415

　　미수금 (주4,7,28)

6,633,248

6,149,209

4,497,257

　　선급비용

3,366,130

2,867,823

2,336,252

　　재고자산 (주8)

51,625,874

52,187,866

41,384,404

　　기타유동자산 (주4,28)

5,038,838

6,316,834

5,081,665

　　매각예정분류자산 (주33)

217,864

0

0

　비유동자산

259,969,423

229,953,926

208,457,973

　　기타포괄손익-공정가치 측정 비유동금융자산 (주4,6,28)

7,481,297

11,397,012

13,965,839

　　당기손익-공정가치금융자산 (주4,6,28)

1,431,394

1,405,468

1,525,344

　　관계종속기업투자자산-지분법 (주9)

11,767,444

10,893,869

8,932,251

　　유형자산 (주10)

187,256,262

168,045,388

149,928,539

　　무형자산 (주11)

22,741,862

20,217,754

20,236,244

　　순확정급여자산 (주14)

4,905,219

5,851,972

2,809,590

　　이연법인세자산 (주25)

10,211,797

5,101,318

4,261,214

　　기타비유동자산 (주4,7,28)

14,174,148

7,041,145

6,798,952

　자산총계

455,905,980

448,424,507

426,621,158

부채

　	　	　
　유동부채

75,719,452

78,344,852

88,117,133

　　매입채무 (주4,28)

11,319,824

10,644,686

13,453,351

　　단기차입금 (주4,5,12,28)

7,114,601

5,147,315

13,687,793

　　미지급금 (주4,28)

15,324,119

17,592,366

15,584,866

　　선수금 (주17)

1,492,602

1,314,934

1,224,812

　　예수금 (주4,28)

892,441

1,298,244

1,294,052

　　미지급비용 (주4,17,28)

26,013,273

29,211,487

27,928,031

　　당기법인세부채

3,358,715

4,250,397

6,749,149

　　유동성장기부채 (주4,12,13,28)

1,308,875

1,089,162

1,329,968

　　충당부채 (주15)

6,524,876

5,844,907

5,372,872

　　기타 유동부채 (주4,17,28)

2,308,472

1,951,354

1,492,239

　　매각예정분류부채 (주33)

61,654

0

0

　비유동부채

16,508,663

15,330,051

33,604,094

　　사채 (주4,13,28)

537,618

536,093

508,232

　　장기차입금 (주4,12,28)

3,724,850

3,560,672

2,866,156

　　장기미지급금 (주4,28)

5,488,283

2,753,305

2,991,440

　　순확정급여부채 (주14)

456,557

268,370

465,884

　　이연법인세부채 (주25)

620,549

5,111,332

23,198,205

　　장기충당부채 (주15)

2,878,450

1,928,518

2,306,994

　　기타 비유동 부채 (주4,17,28)

2,802,356

1,171,761

1,267,183

　부채총계

92,228,115

93,674,903

121,721,227

자본

　	　	　
　지배기업 소유주지분

353,233,775

345,186,142

296,237,697

　　자본금 (주18)

897,514

897,514

897,514

　　　우선주자본금

119,467

119,467

119,467

　　　보통주자본금

778,047

778,047

778,047

　　주식발행초과금

4,403,893

4,403,893

4,403,893

　　이익잉여금 (주19)

346,652,238

337,946,407

293,064,763

　　기타자본항목 (주20,33)

1,280,130

1,938,328

(2,128,473)

　비지배지분 (주31)

10,444,090

9,563,462

8,662,234

　자본총계

363,677,865

354,749,604

304,899,931

자본과부채총계

455,905,980

448,424,507

426,621,158
'''

In [81]:
# 템플릿 정의: 모델에게 질문과 재무제표에 대한 답변을 요청하는 형식
template = '''
You are a helpful assistant. 아래 질문에 맞게 한국어로 답하여라.

question:
{question}

financial_statement:
{financial_statement}

format:
{format}
'''
# 템플릿을 기반으로 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# Pydantic 모델 정의: 재무제표의 요약 정보를 포함하는 데이터 구조
class FinancialStatementSummary(BaseModel):
    asset: str = Field(description='자산')      # 자산 필드
    capital: str = Field(description='자본')    # 자본 필드
    liability: str = Field(description='부채')   # 부채 필드

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,              # 응답의 다양성을 조절하는 온도
)

# PydanticOutputParser 설정: 모델의 출력을 Pydantic 객체로 변환
parser = PydanticOutputParser(pydantic_object=FinancialStatementSummary)

# 프롬프트에 포맷 지침 추가
prompt = prompt.partial(format=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하여 하나의 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 질문과 재무제표를 기반으로 응답 요청
response = chain.invoke({
    'question': '주어진 재무제표에서 핵심 사항을 요약하라.',  # 질문
    'financial_statement': financial_statement  # 주어진 재무제표 데이터
})

# 응답 출력
print(response)

asset=455905980 capital=363677865 liability=92228115


In [84]:
email = '''
Dear Mentors,

I hope you're all doing fantastic!

We're thrilled to share with you the Q3 2024 Mentors' Mastery threads, now conveniently compiled into one PDF file and attached to this email. Dive in, explore, and enjoy your favorite threads. We believe you'll find them engaging and valuable! 🎉

Please stay tuned for upcoming threads this month and next, which will complete the Emotional Intelligence series, with discussions on Mindfulness and Communication topics, as well as the Black Box strategy.

Thank you for your continued dedication and exceptional work. Let’s keep the momentum going and continue to make a positive impact together!

Best regards,

--
Islam Al.Afifi
Sr. Technical Performance Specialist
islam.alafifi@udacity.com  |  udacity.com
'''

## 수신자, 발신자, 발신인의 이메일 주소, 발신자의 직함, 메일 요약

In [85]:
# 템플릿 정의: 모델에게 질문과 이메일 내용을 기반으로 답변을 요청하는 형식
template = '''
You are a helpful assistant. 아래 질문에 맞게 한국어로 답하여라.

question:
{question}

email:
{email}

format:
{format}
'''
# 템플릿을 기반으로 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# Pydantic 모델 정의: 이메일 요약 정보를 포함하는 데이터 구조
class EmailSummary(BaseModel):
    receiver: str = Field(description='수신인')                # 수신인 필드
    sender: str = Field(description='발신인')                 # 발신인 필드
    email: str = Field(description='발신인의 이메일 주소')    # 발신인 이메일 필드
    title: str = Field(description='발신인의 직함')           # 발신인 직함 필드
    summary: str = Field(description='메일 본문 요약')         # 메일 본문 요약 필드

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,              # 응답의 다양성을 조절하는 온도
)

# PydanticOutputParser 설정: 모델의 출력을 Pydantic 객체로 변환
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# 프롬프트에 포맷 지침 추가
prompt = prompt.partial(format=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하여 하나의 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 질문과 이메일 내용을 기반으로 응답 요청
response = chain.invoke({
    'question': '주어진 메일에서 핵심 사항을 요약하라.',  # 질문
    'email': email,  # 주어진 이메일 데이터
})

# 응답 출력
print(response)

receiver='Mentors' sender='Islam Al.Afifi' email='islam.alafifi@udacity.com' title='Sr. Technical Performance Specialist' summary='2024년 3분기 멘토 마스터리 주제가 PDF 파일로 첨부되었습니다. 주제는 감정 지능 시리즈로, 마인드풀니스와 커뮤니케이션에 대한 논의를 포함합니다. 앞으로의 내용도 기대해 주세요.'


### CommaSeparatedListOutputParser

출력 데이터를 ,로 구분된 형식으로 받고 싶을 때 사용

<br>

> ```python
> parser = CommaSeparatedListOutputParser()
> format_instructions = oparser.get_format_instructions()
> prompt = PromptTemplate(
>     template="{subject}에 대해서 대표적인 세 가지 기술.\n{format_instructions}",
>     input_variables=['subject'],
>     partial_variables={"format_instructions": format_instructions},
> )
> 
> model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
> 
> response = chain.invoke(
>     {
>         "financial_statement": financial_statement,
>         "question": "재무제표에서 핵심 사항을 요약하라.",
>     }
> )
> ```

In [87]:
# 템플릿 정의: 모델에게 질문과 재무제표를 제공하고, 지정된 형식으로 답변을 요청하는 구조
template = '''
You are a helpful assistant. 아래 질문에 맞게 한국어로 답하여라.

question:
{question}

financial_statement:
{financial_statement}

format:
{format}
'''
# 템플릿을 기반으로 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,              # 응답의 다양성을 조절하는 온도
)

# 출력 파서 설정: 응답을 콤마로 구분된 목록으로 변환하는 파서
parser = CommaSeparatedListOutputParser()

# 프롬프트에 포맷 지침 추가
prompt = prompt.partial(format=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하여 하나의 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 질문과 재무제표를 기반으로 응답 요청
response = chain.invoke({
    'question': '주어진 재무제표 55기에서 숫자가 가장 큰 계정 다섯 개를 추출.',  # 질문
    'financial_statement': financial_statement  # 주어진 재무제표 데이터
})

# 응답 출력
print(response)

['자산총계', '지배기업 소유주지분', '유동자산', '비유동자산', '자본총계']


In [88]:
# 템플릿 정의: 모델에게 질문과 이메일 내용을 제공하고, 지정된 형식으로 답변을 요청하는 구조
template = '''
You are a helpful assistant. 아래 질문에 맞게 한국어로 답하여라.

question:
{question}

email:
{email}

format:
{format}
'''
# 템플릿을 기반으로 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 모델 설정
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델 이름
    temperature=1,              # 응답의 다양성을 조절하는 온도
)

# 출력 파서 설정: 응답을 콤마로 구분된 목록으로 변환하는 파서
parser = CommaSeparatedListOutputParser()

# 프롬프트에 포맷 지침 추가
prompt = prompt.partial(format=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하여 하나의 체인 구성
chain = prompt | model | parser

# 체인을 사용하여 질문과 이메일 내용을 기반으로 응답 요청
response = chain.invoke({
    'question': '주어진 메일에서 키워드 세 개를 추출하라.',  # 질문
    'email': email,  # 주어진 이메일 데이터
})

# 응답 출력
print(response)

['Q3 2024', 'Emotional Intelligence', 'Mindfulness']


### StructuredOutputParser

답변을 json 형식으로 구성하고 key-value 쌍으로 갖는 여러 필드를 반환하고자 할 때 유용 <br>
이는 로컬 모델이나 parameter 수가 적은 모델에서 유용. <br>
&nbsp;&nbsp;&nbsp;&nbsp; 로컬 모델은 pydantic을 지원하지 않을 가능성이 큼

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> response_schemas = [
>     ResponseSchema(
>         name='date',
>         description='사건이 일어난 날짜.',
>     ),
>     ResponseSchema(
>         name='detail',
>         description='어떠한 사건이 발생하였는지 기술.',
>     ),
> ]
> parser = StructuredOutputParser.from_response_schemas(response_schemas)
> 
> format_instructions = output_parser.get_format_instructions()
> prompt = PromptTemplate(
>     template='가능한 자세하게 질문에 대한 답변.\n{format_instructions}\n{question}',
>     input_variables=['question'],
>     partial_variables={'format_instructions': format_instructions},
> )
> 
> model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
> 
> chain.invoke({"question": "100년전 일어난 역사적인 사건을 알려줄래?"})
> ```

### JsonOutputParser

JSON 스키마를 지정할 수 있게 해주며, 그 스키마에 맞게 결과 전달. <br>

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> parser = JsonOutputParser()
> 
> format_instructions = parser.get_format_instructions()
> prompt = PromptTemplate(
>     template='주어진 질문에 대하여 가능한 상세히 답변.\n{format_instructions}\n{question}',
>     input_variables=['question'],
>     partial_variables={'format_instructions': format_instructions},
> )
> 
> model = model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
> 
> chain.invoke({"question": "100년 전에 일어난 역사적인 사건에 대해서 알려줘. 일어난 날짜는 'date'에, 자세한 사건의 내용은 'event'에 담아줘."})
> ```

### PandasDataFrameOutputParser

임의의 DataFrame을 지정하고 해당 DataFrame에서 데이터를 추출하여 형식화된 사전 형태로 데이터를 조회할 수 있도록 함. <br>


> ```python
> df = pd.read_csv('')
> 
> parser = PandasDataFrameOutputParser(dataframe=df)
> 
> prompt = PromptTemplate(
>     template='유저의 질문에 정확하게 답하여라.\n{format_instructions}\n{query}\n',
>     input_variables=['query'],
>     partial_variables={
>         'format_instructions': parser.get_format_instructions()
>     },
> )
> 
> model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
>
> output = chain.invoke({'query': 'Age column 조회'})
> ```

### DatetimeOutputParser

datetime 형식으로 파싱하는 데 사용

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> parser = DatetimeOutputParser()
> parser.format = "%Y-%m-%d"
> 
> template = """유저의 질문에 대답하라.:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""
> 
> prompt = PromptTemplate.from_template(
>     template,
>     partial_variables={
>         "format_instructions": parser.get_format_instructions()
>     },
> )
> 
> model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
>
> output = chain.invoke({"question": "조선 건국 연도는?"})
> output.strftime("%Y-%m-%d")
> ```

### OutputFixingParser

출력 파싱 과정에서 발생할 수 있는 오류를 자동으로 수정하는 기능을 제공. <br>
기본적으로 다른 parser를 wrapping하고, 해당 parser가 처리할 수 없는 형식의 출력이나 오류를 반환할 경우, 다시 LLM을 호출하여 해당 오류를 수정하도록 설계. <br>

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> class Singer(BaseModel):
>     name: str = Field(description="가수 이름")
>     album: list[str] = Field(description="앨범 리스트")
> 
> parser = PydanticOutputParser(pydantic_object=Singer)
> 
> # 모델 결과가 아래와 같다고 가정
> bad_response = "{'name': '아이유', 'film_names': '너랑나'}"
> parser.parse(bad_response)
>
> new_parser = OutputFixingParser.from_llm(
>     parser=parser,
>     llm=ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> )
> good_response = new_parser.parse(bad_response)
> ```

## Model

### Save & Load

<font style="font-size:20px"> Save </font>

Serialization: 모델을 저장 가능하도록 변경하는 과정. <br>

<br>

> ```python
> dumps(chain)    # json 문자열로 직렬화
> dumpd(chain)    # json으로 직렬화
> 
> with open('<file_name>.pkl', 'wb') as file:
>     pickle.dump(dumpd, file)
> 
> with open("<file_name>.json", "r") as file:
>     json_chain = json.load(file)
>     chain = load(json_chain)
> ```

<br>
<br>

<font style="font-size:20px"> Load </font>

<br>

> ```python
> with open('<file_name>.pkl', 'rb') as file:
>     loaded_chain = pickle.load(file)
> chain = load(loaded_chain)
> 
> chain = load(
>    loaded_chain, secrets_map={'OPENAI_API_KEY': os.environ['OPENAI_API_KEY']}
> )
> with open('<file_name>.json', 'r') as file:
>     json_chain = json.load(file)
>     chain = load(json_chain)
> ```

### Gemini

site: https://aistudio.google.com/app/apikey?hl=ko

<br>

> ```python
> llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
> answer = llm.stream("자연어처리에 대해서 간략히 설명해 줘")
> ```

### Huggingface

<font style="font-size:20px"> Huggingface Endpoint </font>

> ```python
> template = """다음 질문에 한국어로 대답하라.
> #Question: 
> {question}
> 
> #Answer: """
> prompt = PromptTemplate.from_template(template)
> 
> repo_id = "meta-llama/Llama-3.2-1B-Instruct"
> 
> llm = HuggingFaceEndpoint(
>     repo_id=repo_id,
>     max_new_tokens=256,
>     temperature=0.1,
>     huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
> )
>
> chain = prompt | llm | StrOutputParser()
> response = chain.invoke({"question": "what is the capital of South Korea?"})
> print(response)
> ```

<br>
<br>

<font style="font-size:20px"> Local </font>

> ```python
> template = """다음 질문에 한국어로 대답하라.
> #Question: 
> {question}
> 
> #Answer: """
> prompt = PromptTemplate.from_template(template)
> 
> repo_id = "meta-llama/Llama-3.2-1B-Instruct"
> 
> llm = HuggingFacePipeline.from_model_id(
>     model_id=repo_id,
>     task='text-generation',
>     device=0,   # 사용할 device 번호 지정, 지우면 cpu, auto: accelerate 사용
>     pipeline_kwargs={
>        'max_new_tokens': 512,
>     }
> )
> 
> chain = prompt | llm | StrOutputParser()
> response = chain.invoke({"question": "what is the capital of South Korea?"})
> print(response)
> ```

### ollama

> ```python
> llm = ChatOllama(
>     model="llama3.2:1b-instruct-16fp",
> )
> answer = llm.stream("자연어처리에 대해서 간략히 설명해 줘")
> ```